<a href="https://colab.research.google.com/github/Cob-Lim/Tournament-Simulations/blob/main/Compiled_Tournament_Simulations_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameters

In [ ]:
elo_values = [1357.31685295, 1414.50733609, 1452.41032121, 1484.48895542, 1515.18577373, 1547.08778821, 1585.06778005, 1642.12901191] # change as needed
num_of_simulations = 100000 # change as needed

# Importing the Necessary Packages

In [ ]:
import math, random
import numpy as np
import itertools
import matplotlib.pyplot as plt
import pickle # file saving and accessing
import pandas as pd
from scipy.stats import entropy, chisquare, wasserstein_distance
import time

# Tournament Classes

### Player Class

In [ ]:
# Creating a class for a player that has a player number, elo, number of wins, match history, match score, and tiebreaker score
class Player:

    def __init__(self, player_number, elo, wins = 0, losses = 0, match_history = [], match_score = 0, tiebreaker_score = 0):
        self.player_number = player_number
        self.elo = elo
        self.wins = wins
        self.losses = losses
        self.match_history = match_history
        self.match_score = match_score
        self.tiebreaker_score = tiebreaker_score

    def get_player_number(self):
        return self.player_number

    def get_elo(self):
        return self.elo

    def get_wins(self):
        return self.wins

    def get_losses(self):
        return self.losses

    def get_match_history(self):
        return self.match_history

    def get_match_score(self):
        return self.match_score

    def get_tiebreaker_score(self):
        return self.tiebreaker_score

    def set_wins(self, new_wins):
        self.wins = new_wins

    def set_losses(self, new_losses):
        self.losses = new_losses

    def set_match_history(self, new_opponent):
        self.match_history = new_opponent

    def set_match_score(self, new_match_score):
        self.match_score = new_match_score

    def set_tiebreaker_score(self, new_tiebreaker_score):
        self.tiebreaker_score = new_tiebreaker_score

    def __str__(self):
        return f"Player {self.player_number} - Elo: {self.elo} - Wins: {self.wins} - Losses: {self.losses}"

### Round Robin Tournament Class

can be Single Round Robin or Double Round Robin

In [ ]:
# Creating the Round Robin Tournament format
class RoundRobinTournament:

    def __init__(self, players, one_or_two):
        self.players = players # from the list of players created using the Player class
        self.one_or_two = one_or_two # to specify whether a single or double round robin will be used
        self.list_of_player_numbers = [i + 1 for i in range(len(players))] # creates a list of player numbers

    # Function to simulate the tournament
    def simulate_the_tournament(self):

        # Overall flow of the tournament
        #round_counter = 0

        for fighting in range(self.one_or_two):
            for i in range(len(self.players)):
                for j in range(i + 1, len(self.players)):

                  #player_1_person = self.players[i]
                  #player_2_person = self.players[j]

                  #round_counter += 1

                  #print(f"Round {round_counter}")

                  #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

                  self.simulate_match(self.players[i], self.players[j])

        #print("Final Results")

        # Display the interim standing of the players
        #self.display_final_standings()

        ranking_list = self.get_ranking_list()

        return ranking_list

    # Simulating a match for a pair
    def simulate_match(self, player1, player2):

        # Calculate the Elo difference between the players
        elo_difference = player2.get_elo() - player1.get_elo()

        # Calculate the probability of player1 winning using a logistic function (used in chess)
        probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

        # Generate the outcome of the match between a pair
        result = random.random() < probability_player1_win

        # Update wins, losses, and tiebreaker score of the players
        if result:
            #print(f"Player {player1.get_player_number()} wins!")
            player1.set_wins(player1.get_wins() + 1)
            player2.set_losses(player2.get_losses() + 1)
            player1.set_match_score(player1.get_match_score() + 1)
        else:
            #print(f"Player {player2.get_player_number()} wins!")
            player2.set_wins(player2.get_wins() + 1)
            player1.set_losses(player1.get_losses() + 1)
            player2.set_match_score(player2.get_match_score() + 1)

        #print("\n")

    # If you want to see the final standing of players
    def display_final_standings(self):

        sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

        for player in sorted_players:
            print(f"Player {player.player_number} - Elo: {player.elo} - Wins: {player.wins} - Losses: {player.losses} - Match Score: {player.match_score}")

        print("\n")

    # Get the ranking of players for one simulation
    def get_ranking_list(self):

        sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

        player_numbers = [player.get_player_number() for player in sorted_players]

        #print("Player Ranking in Descending Order:")
        #print(player_numbers)
        #print("\n")

        return player_numbers

### Swiss Tournament Class
with either random, Dutch, Burstein, or Monrad pairing

In [ ]:
# Creating the Swiss System Tournament format
class SwissTournament:

    def __init__(self, players, pairing_method, rounds):
        self.players = players # from the list of players created using the Player class
        self.pairing_method = pairing_method # can be random, Dutch, Burstein, Monrad, or Random
        self.score_groups = self.create_score_groups() # creates a dictionary of score groups and the players in it
        self.rounds = rounds # number of rounds for the tournament

    # Function to simulate the tournament
    def simulate_the_tournament(self, pairing_method, rounds):

        # Overall flow of the tournament
        for fighting in range(rounds):

            #print(f"Round {fighting + 1}\n")

            self.create_score_groups()
            #self.display_score_groups()
            self.generate_pairings(pairing_method)
            #self.display_pairings()
            self.simulate_round()

        #print("Match History")

        # Display the match histories and Buchholz score of the players
        self.buchholz_score()
        #self.display_history()

        #print("Final Results")

        # Display the interim standing of the players
        #self.display_final_standings()

        ranking_list = self.get_ranking_list()

        return ranking_list

    # Creating the possible score groups throughout the rounds and placing the players in the appropriate score group
    def create_score_groups(self):
        groups = {}  # a dictionary that maps a score group to the players that belong in it

        for player in self.players:
            score = (player.get_wins(), player.get_losses())  # creates a tuple of wins and losses for the score group

            if score not in groups:
                groups[score] = [player]  # creates a new score group with the given score and adds the player
            else:
                groups[score].append(player)  # adds the player to the existing score group based on the given score

        return groups

    # If you want to see the current standing of players after a round
    def display_score_groups(self):
        for score_group, score_players in self.create_score_groups().items():

            sorted_players = sorted(score_players, key = lambda x: x.get_elo(), reverse=True) # sorted based on elo only

            print(f"Score Group {score_group}:")

            for player in sorted_players:
                print(player)

            print("\n")

    # Generate the pairings for a round (can be Dutch, Bursterin, Monrad, or Random)
    def generate_pairings(self, pairing_method):
        if pairing_method == "Dutch":
            self.pairings = self.dutch_pairing()
        elif pairing_method == "Burstein":
            self.pairings = self.burstein_pairing()
        elif pairing_method == "Monrad":
            self.pairings = self.monrad_pairing()
        elif pairing_method == "Random":
            self.pairings = self.random_pairing()
        else:
            raise ValueError("Invalid pairing method.")

    # If you want to see the pairing of players for the current round
    def display_pairings(self):
        print("Pairings:")

        for player1, player2 in self.pairings:

            if player1 is None:
                print(f"Player {player2.player_number} - Elo: {player2.elo} vs None")
            elif player2 is None:
                print(f"Player {player1.player_number} - Elo: {player1.elo} vs None")
            else:
                print(f"Player {player1.player_number} - Elo: {player1.elo} vs Player {player2.player_number} - Elo: {player2.elo}")

        print("\n")

    # Simulating a match for all the pairings
    def simulate_round(self):

        for player1, player2 in self.pairings:

            # Simulate the battle between two players
            if player1 is None:
                player2.set_wins(player2.get_wins() + 1) # updates the wins
                player2.set_match_score(player2.get_match_score() + 0.5) # updates the tiebreaker score
            elif player2 is None:
                player1.set_wins(player1.get_wins() + 1) # updates the wins
                player1.set_match_score(player1.get_match_score() + 0.5) # updates the tiebreaker score
            else:
                # Calculate the Elo difference between the players
                elo_difference = player2.get_elo() - player1.get_elo()

                # Calculate the probability of player1 winning using a logistic function (used in chess)
                probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

                # Generate the outcome of the match between a pair
                result = random.random() < probability_player1_win

                # Update wins, losses, and tiebreaker score of the players
                if result: # Player 1 wins
                    player1.set_wins(player1.get_wins() + 1)
                    player2.set_losses(player2.get_losses() + 1)
                    player1.set_match_score(player1.get_match_score() + 1)
                else: # Player 2 wins
                    player2.set_wins(player2.get_wins() + 1)
                    player1.set_losses(player1.get_losses() + 1)
                    player2.set_match_score(player2.get_match_score() + 1)

    # Calculate the Buchholz score of each player for tiebreakers
    def buchholz_score(self):

      for curr_player in self.players:

          temp_score = 0 # temporary score of Buccholz score

          for opp_number in curr_player.get_match_history():
            temp_score += opp_number.get_match_score()

          curr_player.set_tiebreaker_score(temp_score)

    # If you want to see the match history and Buchholz score of players
    def display_history(self):
        sorted_players = sorted(self.players, key = lambda x: (x.get_wins(), x.get_losses(), x.get_elo()), reverse=True)

        for player in sorted_players:
            print(f"Player {player.player_number} Match history:")

            for x in player.get_match_history():
              print(x)

        print("\n")

    # If you want to see the final standing of players
    def display_final_standings(self):
        sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

        for player in sorted_players:
            print(f"Player {player.player_number} - Elo: {player.elo} - Wins: {player.wins} - Losses: {player.losses} - Match Score: {player.match_score} - Bachholz Score: {player.tiebreaker_score}")

        print("\n")

    # Get the ranking of players for one simulation
    def get_ranking_list(self):
        sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

        player_numbers = [player.get_player_number() for player in sorted_players]

        #print("Player Ranking in Descending Order:")
        #print(player_numbers)
        #print("\n")

        return player_numbers

    # The Dutch system pairing
    # In this system, separates each score group into an upper half and a lower half
    # Then, the kth-ranked player in the upper half is paired with the kth-ranked player in the lower half
    def dutch_pairing(self):

        pairings = []

        # Iterate through each score group and its corresponding list of players
        for score_group, score_players in self.create_score_groups().items():

            # Sort the players in a score group in descending order based on elo
            sorted_players = sorted(score_players, key = lambda x: x.get_elo(), reverse = True)

            # Checks if the number of players is odd (for the bye system)
            if len(sorted_players) % 2 != 0:
                bye_player = sorted_players[len(sorted_players) // 2] # the player with the median elo in the score group gets a bye
                sorted_players.remove(bye_player)
                pairings.append((bye_player, None))  # add the bye player to the pairings with a "None" opponent
                #bye_player.set_match_history(None) # update match history for the bye player

            # Pair players based on Dutch pairing
            paired_players = []

            for a in range(len(sorted_players) // 2):
                paired_players.append((sorted_players[a], sorted_players[a + len(sorted_players) // 2]))

            # Update match history for paired players
            for player1, player2 in paired_players:

                current_round_history_player1 = [player2]
                current_round_history_player2 = [player1]

                current_round_history_player1 += player1.get_match_history()
                current_round_history_player2 += player2.get_match_history()

                player1.set_match_history(current_round_history_player1)
                player2.set_match_history(current_round_history_player2)

            # Extend the list of pairings with the newly created pairs (to other score groups)
            pairings.extend(paired_players)

        # Return the list of pairings (tuples) for all score groups
        return pairings

    # The Burstein system pairing
    # In this system, the highest-ranked unpaired player is matched with the lowest-ranked unpaired player
    # repeatedly until all players in a score group have a pair
    def burstein_pairing(self):

        pairings = []

        # Iterate through each score group and its corresponding list of players
        for score_group, score_players in self.create_score_groups().items():

            # Sort the players in a score group in descending order based on elo
            sorted_players = sorted(score_players, key = lambda x: x.get_elo(), reverse = True)

            # Checks if the number of players is odd (for the bye system)
            if len(sorted_players) % 2 != 0:
                bye_player = sorted_players[len(sorted_players) // 2] # the player with the median elo in the score group gets a bye
                sorted_players.remove(bye_player)
                pairings.append((bye_player, None))  # add the bye player to the pairings with a "None" opponent
                #bye_player.set_match_history(None) # update match history for the bye player

            # Pair players based on Burstein pairing
            paired_players = []

            for a in range(len(sorted_players) // 2):
              paired_players.append((sorted_players[a], sorted_players[-1 * (a + 1)]))

            # Update match history for paired players
            for player1, player2 in paired_players:

                current_round_history_player1 = [player2]
                current_round_history_player2 = [player1]

                current_round_history_player1 += player1.get_match_history()
                current_round_history_player2 += player2.get_match_history()

                player1.set_match_history(current_round_history_player1)
                player2.set_match_history(current_round_history_player2)

            # Extend the list of pairings with the newly created pairs (to other score groups)
            pairings.extend(paired_players)

        # Return the list of pairings (tuples) for all score groups
        return pairings

    # The Monrad system pairing
    # Players in a score group are first sorted in ascending order by their ranking
    # Then, each unranked player will be paired with the next highest-ranked player
    def monrad_pairing(self):

        pairings = []

        # Iterate through each score group and its corresponding list of players
        for score_group, score_players in self.create_score_groups().items():

            # Sort the players in a score group in descending order based on elo
            sorted_players = sorted(score_players, key = lambda x: x.get_elo(), reverse = True)

            # Checks if the number of players is odd (for the bye system)
            if len(sorted_players) % 2 != 0:
                bye_player = sorted_players[len(sorted_players) // 2] # the player with the median elo in the score group gets a bye
                sorted_players.remove(bye_player)
                pairings.append((bye_player, None))  # add the bye player to the pairings with a "None" opponent
                #bye_player.set_match_history(None) # update match history for the bye player

            # Pair players based on Monrad pairing
            paired_players = []

            for i in range(0, len(sorted_players), 2):
                paired_players.append((sorted_players[i], sorted_players[i + 1]))

            # Update match history for paired players
            for player1, player2 in paired_players:

                current_round_history_player1 = [player2]
                current_round_history_player2 = [player1]

                current_round_history_player1 += player1.get_match_history()
                current_round_history_player2 += player2.get_match_history()

                player1.set_match_history(current_round_history_player1)
                player2.set_match_history(current_round_history_player2)

            # Extend the list of pairings with the newly created pairs (to other score groups)
            pairings.extend(paired_players)

        # Return the list of pairings (tuples) for all score groups
        return pairings

    # Random pairing
    def random_pairing(self):

        pairings = []

        # Iterate through each score group and its corresponding list of players
        for score_group, score_players in self.create_score_groups().items():

            # Shuffle the players in a score group randomly
            random.shuffle(score_players)

            # Checks if the number of players is odd (for the bye system)
            if len(score_players) % 2 != 0:
                bye_player = score_players[-1] # a random player in the score group gets a bye
                score_players.remove(bye_player)
                pairings.append((bye_player, None))  # add the bye player to the pairings with a "None" opponent
                #bye_player.set_match_history(None) # update match history for the bye player

            # Pair players randomly
            paired_players = []

            for i in range(0, len(score_players), 2):
                paired_players.append((score_players[i], score_players[i + 1]))

            # Update match history for paired players
            for player1, player2 in paired_players:

                current_round_history_player1 = [player2]
                current_round_history_player2 = [player1]

                current_round_history_player1 += player1.get_match_history()
                current_round_history_player2 += player2.get_match_history()

                player1.set_match_history(current_round_history_player1)
                player2.set_match_history(current_round_history_player2)

            # Extend the list of pairings with the newly created pairs (to other score groups)
            pairings.extend(paired_players)

        # Return the list of pairings (tuples) for all score groups
        return pairings

### Single Elimination and Multilevel Tournament Class
with either random, Dutch, Burstein, or Monrad pairing

In [ ]:
# Creating the Single Elimination (and Multilevel) Tournament Format
class SingleElimMulti:

  def __init__(self, players, single_or_multi):
      self.players = players # from the list of players created using the Player class
      #self.pairing_method = pairing_method # can be random, Dutch, Burstein, Monrad, or Random
      self.single_or_multi = single_or_multi # to specify whether a single elimination or multilevel tournament will be used
      #self.list_of_player_numbers = [i + 1 for i in range(len(players))] # creates a list of player numbers
      #self.generate_pairings(pairing_method) # creates the pairings

  # Function to simulate the tournament
  def simulate_the_tournament(self, single_or_multi):

      tournament_result = []

      # Simulate the Single Elimination Tournament
      if self.single_or_multi == "single":

        self.round_winners = []
        self.winners = self.players

        while len(self.winners) > 1:
          for i in range(0, len(self.winners), 2):

            #player_1_person = self.winners[i]
            #player_2_person = self.winners[i+1]

            #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

            self.simulate_match(self.winners[i], self.winners[i+1])

          #print(f"Round Winners: {self.round_winners}\n")
          self.winners = self.round_winners
          self.round_winners = []

        # Calculate Buchholz score of the players
        self.buchholz_score()

        #print("Final Results")

        # Display the final standing of the players
        #self.display_final_standings()

        ranking_list = self.get_ranking_list()

        return ranking_list

      # Simulate the Multilevel Tournament
      elif self.single_or_multi == "multi":

        # 1st Round
        #print("Round 1\n")
        round_1 = self.multilevel_round(self.players)

        # 2nd Round
        #print("Round 2\n")
        round_2a = self.multilevel_round(round_1[0])
        round_2b = self.multilevel_round(round_1[1])

        # 3rd Round
        #print("Round 3\n")
        round_3a = self.multilevel_round(round_2a[0])
        round_3b = self.multilevel_round(round_2a[1])
        round_3c = self.multilevel_round(round_2b[0])
        round_3d = self.multilevel_round(round_2b[1])

        final_result = [round_3a[0], round_3a[1], round_3b[0], round_3b[1],
                        round_3c[0], round_3c[1], round_3d[0], round_3d[1]]

        for x in final_result:
          tournament_result.append(x[0].get_player_number())

        return tournament_result

  # Simulating a match for a pair
  def simulate_match(self, player1, player2):

      # Update the match history for the players
      current_round_history_player1 = [player2]
      current_round_history_player2 = [player1]

      current_round_history_player1 += player1.get_match_history()
      current_round_history_player2 += player2.get_match_history()

      player1.set_match_history(current_round_history_player1)
      player2.set_match_history(current_round_history_player2)

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")
          #player1.set_wins(player1.get_wins() + 1)
          #player2.set_losses(player2.get_losses() + 1)
          player1.set_match_score(player1.get_match_score() + 1)

          self.round_winners.append(player1)

      else:
          #print(f"Player {player2.get_player_number()} wins!")
          #player2.set_wins(player2.get_wins() + 1)
          #player1.set_losses(player1.get_losses() + 1)
          player2.set_match_score(player2.get_match_score() + 1)

          self.round_winners.append(player2)

      #print("\n")

  # If you want to see the final standing of players
  def display_final_standings(self):

      sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

      for player in sorted_players:
          print(f"Player {player.player_number} - Elo: {player.elo} - Wins: {player.wins} - Losses: {player.losses} - Match Score: {player.match_score} - Buchholz Score: {player.tiebreaker_score}")

      print("\n")

  # Get the ranking of players for one simulation
  def get_ranking_list(self):

      sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

      player_numbers = [player.get_player_number() for player in sorted_players]

      #print("Player Ranking in Descending Order:")
      #print(player_numbers)
      #print("\n")

      return player_numbers

  # Calculate the Buchholz score of each player for tiebreakers
  def buchholz_score(self):

    for curr_player in self.players:

        temp_score = 0 # temporary score of Buccholz score

        for opp_number in curr_player.get_match_history():
          temp_score += opp_number.get_match_score()

        curr_player.set_tiebreaker_score(temp_score)

  # Simulates one multilevel tournament round
  def multilevel_round(self, players):

    winners = []
    losers = []

    for i in range(0, len(players), 2):

      #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

      results = self.simulate_match_multilevel(player1 = players[i], player2 = players[i + 1])

      winners += results[0]
      losers += results[1]

    return winners, losers

  # Simulating a match for a pair in the multilevel tournament
  def simulate_match_multilevel(self, player1, player2):

      winners = []
      losers = []

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")

          # Update the match history for the players
          current_round_history_player1 = [1]
          current_round_history_player2 = [0]

          current_round_history_player1 += player1.get_match_history()
          current_round_history_player2 += player2.get_match_history()

          player1.set_match_history(current_round_history_player1)
          player2.set_match_history(current_round_history_player2)


          winners.append(player1)
          losers.append(player2)

      else:
          #print(f"Player {player2.get_player_number()} wins!")


          # Update the match history for the players
          current_round_history_player1 = [0]
          current_round_history_player2 = [1]

          current_round_history_player1 += player1.get_match_history()
          current_round_history_player2 += player2.get_match_history()

          player1.set_match_history(current_round_history_player1)
          player2.set_match_history(current_round_history_player2)


          winners.append(player2)
          losers.append(player1)

      #print("\n")

      return winners, losers

### Double Elimination Tournament Class
with either random, Dutch, Burstein, or Monrad pairing

In [ ]:
# Creating the Double Elimination Tournament Format
class DoubleElim:

  def __init__(self, players):
      self.players = players # from the list of players created using the Player class
      #self.pairing_method = pairing_method # can be random, Dutch, Burstein, Monrad, or Random
      #self.list_of_player_numbers = [i + 1 for i in range(len(players))] # creates a list of player numbers

  def simulate_the_tournament(self):

      # Simulate the tournament
      self.winners_bracket = []
      self.losers_bracket = []

      # First Round: Separating into winners and losers brackets
      #print("Round 1: Initial Pairings")

      for i in range(0, len(self.players), 2):

        #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

        self.simulate_match_initial(self.players[i], self.players[i + 1])

      #print(f"Winners Bracket: {self.winners_bracket}")
      #print(f"Losers Bracket: {self.losers_bracket}")
      #print("\n")

      # Second Round: 4 Players in Losers Bracket
      #print("Round 2: Losers Bracket")

      for i in range(0, len(self.losers_bracket), 2):

        #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

        results = self.simulate_match_losers(self.losers_bracket[i], self.losers_bracket[i + 1])

        if results: # player 1 wins
          self.losers_bracket[i + 1] = 0
        else: # player 2 wins
          self.losers_bracket[i] = 0

      # Updated losers bracket
      self.losers_bracket = list(filter(lambda a: a != 0, self.losers_bracket))

      #print(f"Updated Losers Bracket: {self.losers_bracket}")
      #print("\n")

      # Third Round: 4 Players in Winners Bracket
      #print("Round 3: Winners Bracket")

      self.new_winners_bracket = []

      for i in range(0, len(self.winners_bracket), 2):

        #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

        self.simulate_match_winners(self.winners_bracket[i], self.winners_bracket[i + 1])

      self.winners_bracket = self.new_winners_bracket

      #print(f"Updated Winners Bracket: {self.winners_bracket}")
      #print(f"Updated Losers Bracket: {self.losers_bracket}")
      #print("\n")

      # Fourth Round: Updated Losers Bracket
      #print("Round 4: Updated Losers Bracket")

      for i in range(0, len(self.losers_bracket), 2):

        #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

        results = self.simulate_match_losers(self.losers_bracket[i], self.losers_bracket[i + 1])

        if results: # player 1 wins
          self.losers_bracket[i + 1] = 0
        else: # player 2 wins
          self.losers_bracket[i] = 0

      # Updated losers bracket
      self.losers_bracket = list(filter(lambda a: a != 0, self.losers_bracket))

      #print(f"Updated Losers Bracket: {self.losers_bracket}")
      #print("\n")

      # Fifth Round: Losers Finals
      #print("Round 5: Losers Finals")

      for i in range(0, len(self.losers_bracket), 2):

        #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

        results = self.simulate_match_losers(self.losers_bracket[i], self.losers_bracket[i + 1])

        if results: # player 1 wins
          self.losers_bracket[i + 1] = 0
        else: # player 2 wins
          self.losers_bracket[i] = 0

      # Updated losers bracket
      self.losers_bracket = list(filter(lambda a: a != 0, self.losers_bracket))

      #print(f"Losers Finals Winner: Player {self.losers_bracket}")
      #print("\n")

      # Sixth Round: Winners Finals
      #print("Round 6: Winners Finals")

      self.new_winners_bracket = []

      for i in range(0, len(self.winners_bracket), 2):

        #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

        self.simulate_match_winners(self.winners_bracket[i], self.winners_bracket[i + 1])

      self.winners_bracket = self.new_winners_bracket

      #print(f"Updated Winners Bracket: {self.winners_bracket}")
      #print(f"Updated Losers Bracket: {self.losers_bracket}")
      #print("\n")

      # Seventh Round: Grand Finals
      #print("Round 7: Grand Finals")

      final_score = 0

      # Update the match history for the players
      current_round_history_player1 = [self.losers_bracket[0]]
      current_round_history_player2 = [self.winners_bracket[0]]

      current_round_history_player1 += self.winners_bracket[0].get_match_history()
      current_round_history_player2 += self.losers_bracket[0].get_match_history()

      self.winners_bracket[0].set_match_history(current_round_history_player1)
      self.losers_bracket[0].set_match_history(current_round_history_player2)

      # Simulate the grand finals
      for i in range(2):
        results = self.simulate_match_finals(self.winners_bracket[0], self.losers_bracket[0])
        if results:
          final_score += 1

      if final_score >= 1: # if player 1 wins
        #print(f"Tournament Winner: {self.winners_bracket[0].get_player_number()}\n")
        #player_1_person.set_wins(player_1_person.get_wins() + 1)
        #player_2_person.set_losses(player_2_person.get_losses() + 1)
        self.winners_bracket[0].set_match_score(self.winners_bracket[0].get_match_score() + 1)

      else: # if player 2 wins
        #print(f"Tournament Winner: {self.losers_bracket[0].get_player_number()}\n")
        #player_2_person.set_wins(player_2_person.get_wins() + 1)
        #player_1_person.set_losses(player_1_person.get_losses() + 1)
        self.losers_bracket[0].set_match_score(self.losers_bracket[0].get_match_score() + 1)

      # Calculate Buchholz score of the players
      self.buchholz_score()

      #print("Final Results")

      # Display the final standing of the players
      #self.display_final_standings()

      ranking_list = self.get_ranking_list()

      return ranking_list

  # Simulating a match for a pair in the first round
  def simulate_match_initial(self, player1, player2):

      # Update the match history for the players
      current_round_history_player1 = [player2]
      current_round_history_player2 = [player1]

      current_round_history_player1 += player1.get_match_history()
      current_round_history_player2 += player2.get_match_history()

      player1.set_match_history(current_round_history_player1)
      player2.set_match_history(current_round_history_player2)

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")
          #player1.set_wins(player1.get_wins() + 1)
          #player2.set_losses(player2.get_losses() + 1)
          player1.set_match_score(player1.get_match_score() + 1)

          self.winners_bracket.append(player1)
          self.losers_bracket.append(player2)

      else:
          #print(f"Player {player2.get_player_number()} wins!")
          #player2.set_wins(player2.get_wins() + 1)
          #player1.set_losses(player1.get_losses() + 1)
          player2.set_match_score(player2.get_match_score() + 1)

          self.winners_bracket.append(player2)
          self.losers_bracket.append(player1)

      #print("\n")

  # Simulating a match for a pair in the losers bracket
  def simulate_match_losers(self, player1, player2):

      # Update the match history for the players
      current_round_history_player1 = [player2]
      current_round_history_player2 = [player1]

      current_round_history_player1 += player1.get_match_history()
      current_round_history_player2 += player2.get_match_history()

      player1.set_match_history(current_round_history_player1)
      player2.set_match_history(current_round_history_player2)

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")
          #player1.set_wins(player1.get_wins() + 1)
          #player2.set_losses(player2.get_losses() + 1)
          player1.set_match_score(player1.get_match_score() + 1)

          return True

      else:
          #print(f"Player {player2.get_player_number()} wins!")
          #player2.set_wins(player2.get_wins() + 1)
          #player1.set_losses(player1.get_losses() + 1)
          player2.set_match_score(player2.get_match_score() + 1)

          return False

      #print("\n")

  # Simulating a match for a pair in the winners bracket
  def simulate_match_winners(self, player1, player2):

      # Update the match history for the players
      current_round_history_player1 = [player2]
      current_round_history_player2 = [player1]

      current_round_history_player1 += player1.get_match_history()
      current_round_history_player2 += player2.get_match_history()

      player1.set_match_history(current_round_history_player1)
      player2.set_match_history(current_round_history_player2)

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")
          #player1.set_wins(player1.get_wins() + 1)
          #player2.set_losses(player2.get_losses() + 1)
          player1.set_match_score(player1.get_match_score() + 1)

          self.new_winners_bracket.append(player1)
          self.losers_bracket.append(player2)

      else:
          #print(f"Player {player2.get_player_number()} wins!")
          #player2.set_wins(player2.get_wins() + 1)
          #player1.set_losses(player1.get_losses() + 1)
          player2.set_match_score(player2.get_match_score() + 1)

          self.new_winners_bracket.append(player2)
          self.losers_bracket.append(player1)

      #print("\n")

  # Simulating a match for a pair in the grand finals
  def simulate_match_finals(self, player1, player2):

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")

          return True

      else:
          #print(f"Player {player2.get_player_number()} wins!")

          return False

      #print("\n")

  # If you want to see the final standing of players
  def display_final_standings(self):
      sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

      for player in sorted_players:
          print(f"Player {player.player_number} - Elo: {player.elo} - Wins: {player.wins} - Losses: {player.losses} - Match Score: {player.match_score} - Buchholz Score: {player.tiebreaker_score}")

      print("\n")

  # Get the ranking of players for one simulation
  def get_ranking_list(self):

      sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

      player_numbers = [player.get_player_number() for player in sorted_players]

      #print("Player Ranking in Descending Order:")
      #print(player_numbers)
      #print("\n")

      return player_numbers

  # Calculate the Buchholz score of each player for tiebreakers
  def buchholz_score(self):

    for curr_player in self.players:

        temp_score = 0 # temporary score of Buccholz score

        for opp_number in curr_player.get_match_history():
          temp_score += opp_number.get_match_score()

        curr_player.set_tiebreaker_score(temp_score)

### Double Round Robin Split Class
with either random, Dutch, Burstein, or Monrad pairing

In [ ]:
# Creating the Double Round Robin Split Tournament format
class RoundRobinSplit:

  def __init__(self, players):
      self.players = players # from the list of players created using the Player class
      #self.pairing_method = pairing_method # can be random, Dutch, Burstein, Monrad, or Random
      self.splits = [] # to generate the splits of the tournament

  # Function to simulate the tournament
  def simulate_the_tournament(self):

      tournament_result = []
      split_results = []
      final_players = []

      # Overall flow of the tournament
      split_1 = self.players[:4]
      split_2 = self.players[4:]

      self.split_1_player_numbers = [player.get_player_number() for player in split_1] # creates a list of player numbers for split 1
      self.split_2_player_numbers = [player.get_player_number() for player in split_2] # creates a list of player numbers for split 2

      self.splits.append(split_1)
      self.splits.append(split_2)

      #print(f"Splits: {self.splits}")

      for current_split in self.splits:

        #print(f"Current Split: {current_split}\n")

        #round_counter = 0

        for fighting in range(1):
            for i in range(len(current_split)):
                for j in range(i + 1, len(current_split)):

                    #round_counter += 1

                    #print(f"Round {round_counter}")

                    #print(f"Player {player_1_person.player_number} - Elo: {player_1_person.elo} vs Player {player_2_person.player_number} - Elo: {player_2_person.elo}\n")

                    self.simulate_match(current_split[i], current_split[j])

        #print("Interim Results")

        # Display the interim standing of the players
        #self.display_final_standings()

      ranking_list = self.get_ranking_list()

      split_results.append(ranking_list[0]) # gets the winner from one split

      if split_results[0] in self.split_1_player_numbers:
        for i in range(1, len(ranking_list)):
          if ranking_list[i] not in self.split_1_player_numbers and len(split_results) == 1:
            split_results.append(ranking_list[i]) # gets the winner from the other split

      else:
        for i in range(1, len(ranking_list)):
          if ranking_list[i] not in self.split_2_player_numbers and len(split_results) == 1:
            split_results.append(ranking_list[i]) # gets the winner from the other split

      #print(f"Split Results: {split_results}\n")

      # Matches the player number to the player class
      for x in split_results:
        for player in self.players:
          if player.player_number == x:
            final_players.append(player)

      # Final Match
      self.simulate_match(final_players[0], final_players[1])

      # Display the interim standing of the players
      #self.display_final_standings()

      ranking_list = self.get_ranking_list()

      tournament_result.append(ranking_list[0])

      #print(f"Winner: {tournament_result[0]}")

      return tournament_result

  # Simulating a match for a pair
  def simulate_match(self, player1, player2):

      #print(f"Player {player1.player_number} - Elo: {player1.elo} vs Player {player2.player_number} - Elo: {player2.elo}\n")

      # Calculate the Elo difference between the players
      elo_difference = player2.get_elo() - player1.get_elo()

      # Calculate the probability of player1 winning using a logistic function (used in chess)
      probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

      # Generate the outcome of the match between a pair
      result = random.random() < probability_player1_win

      # Update wins, losses, and tiebreaker score of the players
      if result:
          #print(f"Player {player1.get_player_number()} wins!")
          #player1.set_wins(player1.get_wins() + 1)
          #player2.set_losses(player2.get_losses() + 1)
          player1.set_match_score(player1.get_match_score() + 1)
      else:
          #print(f"Player {player2.get_player_number()} wins!")
          #player2.set_wins(player2.get_wins() + 1)
          #player1.set_losses(player1.get_losses() + 1)
          player2.set_match_score(player2.get_match_score() + 1)

      #print("\n")

  # If you want to see the final standing of players
  def display_final_standings(self):
      sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

      for player in sorted_players:
          print(f"Player {player.player_number} - Elo: {player.elo} - Wins: {player.wins} - Losses: {player.losses} - Match Score: {player.match_score}")

      print("\n")

  # Get the ranking of players for one simulation
  def get_ranking_list(self):
      sorted_players = sorted(self.players, key=lambda x: (x.get_match_score(), x.get_tiebreaker_score(), x.get_elo()), reverse=True)

      player_numbers = [player.get_player_number() for player in sorted_players]

      #print("Player Ranking in Descending Order:")
      #print(player_numbers)
      #print("\n")

      return player_numbers

### Ladder Tournament Class

In [ ]:
# Creating the Ladder Tournament format
class LadderTournament:

    def __init__(self, players, rounds):
        self.players = players # from the list of players created using the Player class
        self.rounds = rounds # number of rounds for the tournament
        self.tournament_result = [] # creates a list for the winner
        #self.list_of_player_numbers = [i + 1 for i in range(len(players))] # creates a list of player numbers
        self.fight = []  # stores the match for each round


    # Function to simulate the tournament
    def simulate_the_tournament(self, rounds):

        # Overall flow of the tournament
        for fighting in range(rounds):

            #print(f"Round {fighting + 1}\n")

            self.generate_match()
            self.simulate_round()

        #print(f"The winner of the tournament is Player {self.tournament_result[-1].get_player_number()}")

        return [self.tournament_result[-1].get_player_number()]

    # Generate the match for the round
    def generate_match(self):

        # To determine the players participating in the match
        self.fight = [] # resets the match list for each round

        if len(self.tournament_result) == 0:
            player1 = random.choice(self.players) # chooses a player at random
            self.players.remove(player1) # removes the player from the possible choices
        else:
            player1 = self.tournament_result[0] # gets the current winner of the tournament

        player2 = random.choice(self.players) # chooses another player at random
        self.players.remove(player2) # removes the player from the possible choices

        #print(f"Player {player1.player_number} - Elo: {player1.elo} vs Player {player2.player_number} - Elo: {player2.elo}\n")

        self.fight.append((player1, player2)) # adds players 1 and 2 to the match list

    # Simulating a match for all the pairings
    def simulate_round(self):

        for player1, player2 in self.fight:

            # Calculate the Elo difference between the players
            elo_difference = player2.get_elo() - player1.get_elo()

            # Calculate the probability of player1 winning using a logistic function (used in chess)
            probability_player1_win = 1 / (1 + 10 ** (elo_difference / 400.0))

            # Generate the outcome of the match between a pair
            result = random.random() < probability_player1_win

            # Update wins, losses, and tiebreaker score of the players
            if result:
                #print(f"Winner: Player {player1.player_number}\n")
                self.tournament_result.append(player1)
            else:
                #print(f"Winner: Player {player2.player_number}\n")
                self.tournament_result.append(player2)

# Helper Functions

In [ ]:
# Generate the pairings for a round (can be Dutch, Bursterin, Monrad, or Random)
def generate_pairings(list_of_players, pairing_method):

  pairings = []

  if pairing_method == "Dutch":

    # The Dutch system pairing
    # In this system, separates each score group into an upper half and a lower half
    # Then, the kth-ranked player in the upper half is paired with the kth-ranked player in the lower half

    for x in range(len(list_of_players) // 2):
      pairings.append(list_of_players[x])
      pairings.append(list_of_players[x + 4])

  elif pairing_method == "Burstein":

    # The Burstein system pairing
    # In this system, the highest-ranked unpaired player is matched with the lowest-ranked unpaired player
    # repeatedly until all players in a score group have a pair

    for x in range(len(list_of_players) // 2):
      pairings.append(list_of_players[x])
      pairings.append(list_of_players[-1 * (x + 1)])

  elif pairing_method == "Monrad":

    # The Monrad system pairing
    # Players in a score group are first sorted in ascending order by their ranking
    # Then, each unranked player will be paired with the next highest-ranked player

    pairings = list_of_players

  elif pairing_method == "Random":

    # Random pairing
    temp_players = list_of_players[:]

    random.shuffle(temp_players)

    pairings = temp_players

  return pairings

In [ ]:
def generate_players(elo_values, pairing_method):

  #elo_values = np.round(elo_values).astype(int)

  elo_values = sorted(elo_values, reverse = True)

  list_of_player_numbers = sorted([i + 1 for i in range(len(elo_values))], reverse = True)

  if pairing_method == "Dutch":

    elo_values = generate_pairings(elo_values, "Dutch")
    list_of_player_numbers = generate_pairings(list_of_player_numbers, "Dutch")

    players = [Player(player_number = list_of_player_numbers[i - 1], elo = elo) for i, elo in enumerate(elo_values, start = 1)]

  elif pairing_method == "Burstein":

    elo_values = generate_pairings(elo_values, "Burstein")
    list_of_player_numbers = generate_pairings(list_of_player_numbers, "Burstein")

    players = [Player(player_number = list_of_player_numbers[i - 1], elo = elo) for i, elo in enumerate(elo_values, start = 1)]

  elif pairing_method == "Monrad":

    elo_values = generate_pairings(elo_values, "Monrad")
    list_of_player_numbers = generate_pairings(list_of_player_numbers, "Monrad")

    players = [Player(player_number = list_of_player_numbers[i - 1], elo = elo) for i, elo in enumerate(elo_values, start = 1)]

  elif pairing_method == "Random":

    players = [Player(player_number = list_of_player_numbers[i - 1], elo = elo) for i, elo in enumerate(elo_values, start = 1)]

    players = generate_pairings(players, "Random")

  elif pairing_method == "None":

    players = [Player(player_number = list_of_player_numbers[i - 1], elo = elo) for i, elo in enumerate(elo_values, start = 1)]

  return players

In [ ]:
def generate_permutations(subset):

  temp_permutations = itertools.permutations(range(1, 9), subset)
  temp_permutations_tuples = list(map(tuple, temp_permutations))

  ranking_permutations = dict.fromkeys(temp_permutations_tuples, 1)

  return ranking_permutations

# Generate the permutations for top 8 players for copying
permutations_8 = dict(sorted(generate_permutations(8).items(), key = lambda item: item[0], reverse = True))

# Generate the permutations for top 3 players for copying
permutations_3 = dict(sorted(generate_permutations(3).items(), key = lambda item: item[0], reverse = True))

# Generate the permutations for top 1 player for copying
permutations_1 = dict(sorted(generate_permutations(1).items(), key = lambda item: item[0], reverse = True))

In [ ]:
def extract_n_players(simulation_results_list, n_players):

  top_n_players = (*simulation_results_list[:n_players],)

  return top_n_players

# Simulations

### 1) Simulation of Double Round Robin Tournament

In [ ]:
start = time.perf_counter()

double_ranking_counts_8 = permutations_8.copy()

double_ranking_counts_3 = permutations_3.copy()

double_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Double Round Robin tournament with the list of players
    tournament_double_round_robin = RoundRobinTournament(players, one_or_two = 2)

    tournament_result = tournament_double_round_robin.simulate_the_tournament()

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    double_ranking_counts_8[tournament_result_8] += 1
    double_ranking_counts_3[tournament_result_3] += 1
    double_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_double_8 = sum(double_ranking_counts_8.values())
sum_double_3 = sum(double_ranking_counts_3.values())
sum_double_1 = sum(double_ranking_counts_1.values())

# Calculate probabilities
probabilities_double_8 = {rankings: count / sum_double_8 for rankings, count in double_ranking_counts_8.items()}
probabilities_double_3 = {rankings: count / sum_double_3 for rankings, count in double_ranking_counts_3.items()}
probabilities_double_1 = {rankings: count / sum_double_1 for rankings, count in double_ranking_counts_1.items()}

# Save the results to a pickle file
with open('double_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_double_8': probabilities_double_8,
        'probabilities_double_3': probabilities_double_3,
        'probabilities_double_1': probabilities_double_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 26.87 second(s)


### 2) Simulation of Single Round Robin

In [ ]:
start = time.perf_counter()

single_ranking_counts_8 = permutations_8.copy()

single_ranking_counts_3 = permutations_3.copy()

single_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Single Round Robin tournament with the list of players
    tournament_single_round_robin = RoundRobinTournament(players, one_or_two = 1)

    tournament_result = tournament_single_round_robin.simulate_the_tournament()

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    single_ranking_counts_8[tournament_result_8] += 1
    single_ranking_counts_3[tournament_result_3] += 1
    single_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_single_8 = sum(single_ranking_counts_8.values())
sum_single_3 = sum(single_ranking_counts_3.values())
sum_single_1 = sum(single_ranking_counts_1.values())

# Calculate probabilities
probabilities_single_8 = {rankings: count / sum_single_8 for rankings, count in single_ranking_counts_8.items()}
probabilities_single_3 = {rankings: count / sum_single_3 for rankings, count in single_ranking_counts_3.items()}
probabilities_single_1 = {rankings: count / sum_single_1 for rankings, count in single_ranking_counts_1.items()}

# Save the results to a pickle file
with open('single_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_single_8': probabilities_single_8,
        'probabilities_single_3': probabilities_single_3,
        'probabilities_single_1': probabilities_single_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 10.81 second(s)


### 3) Simulation of Swiss Tournament Using the Dutch Pairing System (3 Rounds)

In [ ]:
start = time.perf_counter()

dutch_ranking_counts_8 = permutations_8.copy()

dutch_ranking_counts_3 = permutations_3.copy()

dutch_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Dutch system pairing
    tournament_dutch = SwissTournament(players, pairing_method = "Dutch", rounds = 3)

    tournament_result = tournament_dutch.simulate_the_tournament(tournament_dutch.pairing_method, tournament_dutch.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    dutch_ranking_counts_8[tournament_result_8] += 1
    dutch_ranking_counts_3[tournament_result_3] += 1
    dutch_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_dutch_8 = sum(dutch_ranking_counts_8.values())
sum_dutch_3 = sum(dutch_ranking_counts_3.values())
sum_dutch_1 = sum(dutch_ranking_counts_1.values())

# Calculate probabilities
probabilities_dutch_8 = {rankings: count / sum_dutch_8 for rankings, count in dutch_ranking_counts_8.items()}
probabilities_dutch_3 = {rankings: count / sum_dutch_3 for rankings, count in dutch_ranking_counts_3.items()}
probabilities_dutch_1 = {rankings: count / sum_dutch_1 for rankings, count in dutch_ranking_counts_1.items()}

# Save the results to a pickle file
with open('dutch_simulation_results_3.pickle', 'wb') as file:
    saved_data = {
        'probabilities_dutch_8': probabilities_dutch_8,
        'probabilities_dutch_3': probabilities_dutch_3,
        'probabilities_dutch_1': probabilities_dutch_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 11.5 second(s)


### 4) Simulation of Swiss Tournament Using the Dutch Pairing System (4 Rounds)

In [ ]:
start = time.perf_counter()

dutch_ranking_counts_8 = permutations_8.copy()

dutch_ranking_counts_3 = permutations_3.copy()

dutch_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Dutch system pairing
    tournament_dutch = SwissTournament(players, pairing_method = "Dutch", rounds = 4)

    tournament_result = tournament_dutch.simulate_the_tournament(tournament_dutch.pairing_method, tournament_dutch.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    dutch_ranking_counts_8[tournament_result_8] += 1
    dutch_ranking_counts_3[tournament_result_3] += 1
    dutch_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_dutch_8 = sum(dutch_ranking_counts_8.values())
sum_dutch_3 = sum(dutch_ranking_counts_3.values())
sum_dutch_1 = sum(dutch_ranking_counts_1.values())

# Calculate probabilities
probabilities_dutch_8 = {rankings: count / sum_dutch_8 for rankings, count in dutch_ranking_counts_8.items()}
probabilities_dutch_3 = {rankings: count / sum_dutch_3 for rankings, count in dutch_ranking_counts_3.items()}
probabilities_dutch_1 = {rankings: count / sum_dutch_1 for rankings, count in dutch_ranking_counts_1.items()}

# Save the results to a pickle file
with open('dutch_simulation_results_4.pickle', 'wb') as file:
    saved_data = {
        'probabilities_dutch_8': probabilities_dutch_8,
        'probabilities_dutch_3': probabilities_dutch_3,
        'probabilities_dutch_1': probabilities_dutch_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 14.11 second(s)


### 5) Simulation of Swiss Tournament Using the Dutch Pairing System (5 Rounds)

In [ ]:
start = time.perf_counter()

dutch_ranking_counts_8 = permutations_8.copy()

dutch_ranking_counts_3 = permutations_3.copy()

dutch_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Dutch system pairing
    tournament_dutch = SwissTournament(players, pairing_method = "Dutch", rounds = 5)

    tournament_result = tournament_dutch.simulate_the_tournament(tournament_dutch.pairing_method, tournament_dutch.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    dutch_ranking_counts_8[tournament_result_8] += 1
    dutch_ranking_counts_3[tournament_result_3] += 1
    dutch_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_dutch_8 = sum(dutch_ranking_counts_8.values())
sum_dutch_3 = sum(dutch_ranking_counts_3.values())
sum_dutch_1 = sum(dutch_ranking_counts_1.values())

# Calculate probabilities
probabilities_dutch_8 = {rankings: count / sum_dutch_8 for rankings, count in dutch_ranking_counts_8.items()}
probabilities_dutch_3 = {rankings: count / sum_dutch_3 for rankings, count in dutch_ranking_counts_3.items()}
probabilities_dutch_1 = {rankings: count / sum_dutch_1 for rankings, count in dutch_ranking_counts_1.items()}

# Save the results to a pickle file
with open('dutch_simulation_results_5.pickle', 'wb') as file:
    saved_data = {
        'probabilities_dutch_8': probabilities_dutch_8,
        'probabilities_dutch_3': probabilities_dutch_3,
        'probabilities_dutch_1': probabilities_dutch_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 17.67 second(s)


### 6) Simulation of Swiss Tournament Using the Burstein Pairing System (3 Rounds)

In [ ]:
start = time.perf_counter()

burstein_ranking_counts_8 = permutations_8.copy()

burstein_ranking_counts_3 = permutations_3.copy()

burstein_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Burstein system pairing
    tournament_burstein = SwissTournament(players, pairing_method = "Burstein", rounds = 3)

    tournament_result = tournament_burstein.simulate_the_tournament(tournament_burstein.pairing_method, tournament_burstein.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    burstein_ranking_counts_8[tournament_result_8] += 1
    burstein_ranking_counts_3[tournament_result_3] += 1
    burstein_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_burstein_8 = sum(burstein_ranking_counts_8.values())
sum_burstein_3 = sum(burstein_ranking_counts_3.values())
sum_burstein_1 = sum(burstein_ranking_counts_1.values())

# Calculate probabilities
probabilities_burstein_8 = {rankings: count / sum_burstein_8 for rankings, count in burstein_ranking_counts_8.items()}
probabilities_burstein_3 = {rankings: count / sum_burstein_3 for rankings, count in burstein_ranking_counts_3.items()}
probabilities_burstein_1 = {rankings: count / sum_burstein_1 for rankings, count in burstein_ranking_counts_1.items()}

# Save the results to a pickle file
with open('burstein_simulation_results_3.pickle', 'wb') as file:
    saved_data = {
        'probabilities_burstein_8': probabilities_burstein_8,
        'probabilities_burstein_3': probabilities_burstein_3,
        'probabilities_burstein_1': probabilities_burstein_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 10.3 second(s)


### 7) Simulation of Swiss Tournament Using the Burstein Pairing System (4 Rounds)

In [ ]:
start = time.perf_counter()

burstein_ranking_counts_8 = permutations_8.copy()

burstein_ranking_counts_3 = permutations_3.copy()

burstein_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Burstein system pairing
    tournament_burstein = SwissTournament(players, pairing_method = "Burstein", rounds = 4)

    tournament_result = tournament_burstein.simulate_the_tournament(tournament_burstein.pairing_method, tournament_burstein.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    burstein_ranking_counts_8[tournament_result_8] += 1
    burstein_ranking_counts_3[tournament_result_3] += 1
    burstein_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_burstein_8 = sum(burstein_ranking_counts_8.values())
sum_burstein_3 = sum(burstein_ranking_counts_3.values())
sum_burstein_1 = sum(burstein_ranking_counts_1.values())

# Calculate probabilities
probabilities_burstein_8 = {rankings: count / sum_burstein_8 for rankings, count in burstein_ranking_counts_8.items()}
probabilities_burstein_3 = {rankings: count / sum_burstein_3 for rankings, count in burstein_ranking_counts_3.items()}
probabilities_burstein_1 = {rankings: count / sum_burstein_1 for rankings, count in burstein_ranking_counts_1.items()}

# Save the results to a pickle file
with open('burstein_simulation_results_4.pickle', 'wb') as file:
    saved_data = {
        'probabilities_burstein_8': probabilities_burstein_8,
        'probabilities_burstein_3': probabilities_burstein_3,
        'probabilities_burstein_1': probabilities_burstein_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 15.08 second(s)


### 8) Simulation of Swiss Tournament Using the Burstein Pairing System (5 Rounds)

In [ ]:
start = time.perf_counter()

burstein_ranking_counts_8 = permutations_8.copy()

burstein_ranking_counts_3 = permutations_3.copy()

burstein_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Burstein system pairing
    tournament_burstein = SwissTournament(players, pairing_method = "Burstein", rounds = 5)

    tournament_result = tournament_burstein.simulate_the_tournament(tournament_burstein.pairing_method, tournament_burstein.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    burstein_ranking_counts_8[tournament_result_8] += 1
    burstein_ranking_counts_3[tournament_result_3] += 1
    burstein_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_burstein_8 = sum(burstein_ranking_counts_8.values())
sum_burstein_3 = sum(burstein_ranking_counts_3.values())
sum_burstein_1 = sum(burstein_ranking_counts_1.values())

# Calculate probabilities
probabilities_burstein_8 = {rankings: count / sum_burstein_8 for rankings, count in burstein_ranking_counts_8.items()}
probabilities_burstein_3 = {rankings: count / sum_burstein_3 for rankings, count in burstein_ranking_counts_3.items()}
probabilities_burstein_1 = {rankings: count / sum_burstein_1 for rankings, count in burstein_ranking_counts_1.items()}

# Save the results to a pickle file
with open('burstein_simulation_results_5.pickle', 'wb') as file:
    saved_data = {
        'probabilities_burstein_8': probabilities_burstein_8,
        'probabilities_burstein_3': probabilities_burstein_3,
        'probabilities_burstein_1': probabilities_burstein_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 16.7 second(s)


### 9) Simulation of Swiss Tournament Using the Monrad Pairing System (3 Rounds)

In [ ]:
start = time.perf_counter()

monrad_ranking_counts_8 = permutations_8.copy()

monrad_ranking_counts_3 = permutations_3.copy()

monrad_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Monrad system pairing
    tournament_monrad = SwissTournament(players, pairing_method = "Monrad", rounds = 3)

    tournament_result = tournament_monrad.simulate_the_tournament(tournament_monrad.pairing_method, tournament_monrad.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    monrad_ranking_counts_8[tournament_result_8] += 1
    monrad_ranking_counts_3[tournament_result_3] += 1
    monrad_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_monrad_8 = sum(monrad_ranking_counts_8.values())
sum_monrad_3 = sum(monrad_ranking_counts_3.values())
sum_monrad_1 = sum(monrad_ranking_counts_1.values())

# Calculate probabilities
probabilities_monrad_8 = {rankings: count / sum_monrad_8 for rankings, count in monrad_ranking_counts_8.items()}
probabilities_monrad_3 = {rankings: count / sum_monrad_3 for rankings, count in monrad_ranking_counts_3.items()}
probabilities_monrad_1 = {rankings: count / sum_monrad_1 for rankings, count in monrad_ranking_counts_1.items()}

# Save the results to a pickle file
with open('monrad_simulation_results_3.pickle', 'wb') as file:
    saved_data = {
        'probabilities_monrad_8': probabilities_monrad_8,
        'probabilities_monrad_3': probabilities_monrad_3,
        'probabilities_monrad_1': probabilities_monrad_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 11.46 second(s)


### 10) Simulation of Swiss Tournament Using the Monrad Pairing System (4 Rounds)

In [ ]:
start = time.perf_counter()

monrad_ranking_counts_8 = permutations_8.copy()

monrad_ranking_counts_3 = permutations_3.copy()

monrad_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Monrad system pairing
    tournament_monrad = SwissTournament(players, pairing_method = "Monrad", rounds = 4)

    tournament_result = tournament_monrad.simulate_the_tournament(tournament_monrad.pairing_method, tournament_monrad.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    monrad_ranking_counts_8[tournament_result_8] += 1
    monrad_ranking_counts_3[tournament_result_3] += 1
    monrad_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_monrad_8 = sum(monrad_ranking_counts_8.values())
sum_monrad_3 = sum(monrad_ranking_counts_3.values())
sum_monrad_1 = sum(monrad_ranking_counts_1.values())

# Calculate probabilities
probabilities_monrad_8 = {rankings: count / sum_monrad_8 for rankings, count in monrad_ranking_counts_8.items()}
probabilities_monrad_3 = {rankings: count / sum_monrad_3 for rankings, count in monrad_ranking_counts_3.items()}
probabilities_monrad_1 = {rankings: count / sum_monrad_1 for rankings, count in monrad_ranking_counts_1.items()}

# Save the results to a pickle file
with open('monrad_simulation_results_4.pickle', 'wb') as file:
    saved_data = {
        'probabilities_monrad_8': probabilities_monrad_8,
        'probabilities_monrad_3': probabilities_monrad_3,
        'probabilities_monrad_1': probabilities_monrad_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 14.07 second(s)


### 11) Simulation of Swiss Tournament Using the Monrad Pairing System (5 Rounds)

In [ ]:
start = time.perf_counter()

monrad_ranking_counts_8 = permutations_8.copy()

monrad_ranking_counts_3 = permutations_3.copy()

monrad_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using the Monrad system pairing
    tournament_monrad = SwissTournament(players, pairing_method = "Monrad", rounds = 5)

    tournament_result = tournament_monrad.simulate_the_tournament(tournament_monrad.pairing_method, tournament_monrad.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    monrad_ranking_counts_8[tournament_result_8] += 1
    monrad_ranking_counts_3[tournament_result_3] += 1
    monrad_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_monrad_8 = sum(monrad_ranking_counts_8.values())
sum_monrad_3 = sum(monrad_ranking_counts_3.values())
sum_monrad_1 = sum(monrad_ranking_counts_1.values())

# Calculate probabilities
probabilities_monrad_8 = {rankings: count / sum_monrad_8 for rankings, count in monrad_ranking_counts_8.items()}
probabilities_monrad_3 = {rankings: count / sum_monrad_3 for rankings, count in monrad_ranking_counts_3.items()}
probabilities_monrad_1 = {rankings: count / sum_monrad_1 for rankings, count in monrad_ranking_counts_1.items()}

# Save the results to a pickle file
with open('monrad_simulation_results_5.pickle', 'wb') as file:
    saved_data = {
        'probabilities_monrad_8': probabilities_monrad_8,
        'probabilities_monrad_3': probabilities_monrad_3,
        'probabilities_monrad_1': probabilities_monrad_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 16.42 second(s)


### 12) Simulation of Swiss Tournament Using the Random Pairing System (3 Rounds)

In [ ]:
start = time.perf_counter()

random_ranking_counts_8 = permutations_8.copy()

random_ranking_counts_3 = permutations_3.copy()

random_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using random pairing
    tournament_random = SwissTournament(players, pairing_method = "Random", rounds = 3)

    tournament_result = tournament_random.simulate_the_tournament(tournament_random.pairing_method, tournament_random.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    random_ranking_counts_8[tournament_result_8] += 1
    random_ranking_counts_3[tournament_result_3] += 1
    random_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_random_8 = sum(random_ranking_counts_8.values())
sum_random_3 = sum(random_ranking_counts_3.values())
sum_random_1 = sum(random_ranking_counts_1.values())

# Calculate probabilities
probabilities_random_8 = {rankings: count / sum_random_8 for rankings, count in random_ranking_counts_8.items()}
probabilities_random_3 = {rankings: count / sum_random_3 for rankings, count in random_ranking_counts_3.items()}
probabilities_random_1 = {rankings: count / sum_random_1 for rankings, count in random_ranking_counts_1.items()}

# Save the results to a pickle file
with open('random_simulation_results_3.pickle', 'wb') as file:
    saved_data = {
        'probabilities_random_8': probabilities_random_8,
        'probabilities_random_3': probabilities_random_3,
        'probabilities_random_1': probabilities_random_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 12.33 second(s)


### 13) Simulation of Swiss Tournament Using the Random Pairing System (4 Rounds)

In [ ]:
start = time.perf_counter()

random_ranking_counts_8 = permutations_8.copy()

random_ranking_counts_3 = permutations_3.copy()

random_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using random pairing
    tournament_random = SwissTournament(players, pairing_method = "Random", rounds = 4)

    tournament_result = tournament_random.simulate_the_tournament(tournament_random.pairing_method, tournament_random.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    random_ranking_counts_8[tournament_result_8] += 1
    random_ranking_counts_3[tournament_result_3] += 1
    random_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_random_8 = sum(random_ranking_counts_8.values())
sum_random_3 = sum(random_ranking_counts_3.values())
sum_random_1 = sum(random_ranking_counts_1.values())

# Calculate probabilities
probabilities_random_8 = {rankings: count / sum_random_8 for rankings, count in random_ranking_counts_8.items()}
probabilities_random_3 = {rankings: count / sum_random_3 for rankings, count in random_ranking_counts_3.items()}
probabilities_random_1 = {rankings: count / sum_random_1 for rankings, count in random_ranking_counts_1.items()}

# Save the results to a pickle file
with open('random_simulation_results_4.pickle', 'wb') as file:
    saved_data = {
        'probabilities_random_8': probabilities_random_8,
        'probabilities_random_3': probabilities_random_3,
        'probabilities_random_1': probabilities_random_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 16.03 second(s)


### 14) Simulation of Swiss Tournament Using the Random Pairing System (5 Rounds)

In [ ]:
start = time.perf_counter()

random_ranking_counts_8 = permutations_8.copy()

random_ranking_counts_3 = permutations_3.copy()

random_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Swiss tournament with the list of players and using random pairing
    tournament_random = SwissTournament(players, pairing_method = "Random", rounds = 5)

    tournament_result = tournament_random.simulate_the_tournament(tournament_random.pairing_method, tournament_random.rounds)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    random_ranking_counts_8[tournament_result_8] += 1
    random_ranking_counts_3[tournament_result_3] += 1
    random_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_random_8 = sum(random_ranking_counts_8.values())
sum_random_3 = sum(random_ranking_counts_3.values())
sum_random_1 = sum(random_ranking_counts_1.values())

# Calculate probabilities
probabilities_random_8 = {rankings: count / sum_random_8 for rankings, count in random_ranking_counts_8.items()}
probabilities_random_3 = {rankings: count / sum_random_3 for rankings, count in random_ranking_counts_3.items()}
probabilities_random_1 = {rankings: count / sum_random_1 for rankings, count in random_ranking_counts_1.items()}

# Save the results to a pickle file
with open('random_simulation_results_5.pickle', 'wb') as file:
    saved_data = {
        'probabilities_random_8': probabilities_random_8,
        'probabilities_random_3': probabilities_random_3,
        'probabilities_random_1': probabilities_random_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 18.2 second(s)


### 15) Simulation of Single Elimination Tournament Using the Dutch Pairing System

In [ ]:
start = time.perf_counter()

single_dutch_ranking_counts_8 = permutations_8.copy()

single_dutch_ranking_counts_3 = permutations_3.copy()

single_dutch_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Dutch")

    # Create a Single Elimination tournament with the list of players and using the Dutch system pairing
    single_elim_dutch = SingleElimMulti(players, single_or_multi = "single")

    tournament_result = single_elim_dutch.simulate_the_tournament(single_elim_dutch.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    single_dutch_ranking_counts_8[tournament_result_8] += 1
    single_dutch_ranking_counts_3[tournament_result_3] += 1
    single_dutch_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_single_dutch_8 = sum(single_dutch_ranking_counts_8.values())
sum_single_dutch_3 = sum(single_dutch_ranking_counts_3.values())
sum_single_dutch_1 = sum(single_dutch_ranking_counts_1.values())

# Calculate probabilities
probabilities_single_dutch_8 = {rankings: count / sum_single_dutch_8 for rankings, count in single_dutch_ranking_counts_8.items()}
probabilities_single_dutch_3 = {rankings: count / sum_single_dutch_3 for rankings, count in single_dutch_ranking_counts_3.items()}
probabilities_single_dutch_1 = {rankings: count / sum_single_dutch_1 for rankings, count in single_dutch_ranking_counts_1.items()}

# Save the results to a pickle file
with open('single_elim_dutch_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_single_dutch_8': probabilities_single_dutch_8,
        'probabilities_single_dutch_3': probabilities_single_dutch_3,
        'probabilities_single_dutch_1': probabilities_single_dutch_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 4.46 second(s)


### 16) Simulation of Single Elimination Tournament Using the Burstein Pairing System

In [ ]:
start = time.perf_counter()

single_burstein_ranking_counts_8 = permutations_8.copy()

single_burstein_ranking_counts_3 = permutations_3.copy()

single_burstein_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Burstein")

    # Create a Single Elimination tournament with the list of players and using the Burstein system pairing
    single_elim_burstein = SingleElimMulti(players, single_or_multi = "single")

    tournament_result = single_elim_burstein.simulate_the_tournament(single_elim_burstein.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    single_burstein_ranking_counts_8[tournament_result_8] += 1
    single_burstein_ranking_counts_3[tournament_result_3] += 1
    single_burstein_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_single_burstein_8 = sum(single_burstein_ranking_counts_8.values())
sum_single_burstein_3 = sum(single_burstein_ranking_counts_3.values())
sum_single_burstein_1 = sum(single_burstein_ranking_counts_1.values())

# Calculate probabilities
probabilities_single_burstein_8 = {rankings: count / sum_single_burstein_8 for rankings, count in single_burstein_ranking_counts_8.items()}
probabilities_single_burstein_3 = {rankings: count / sum_single_burstein_3 for rankings, count in single_burstein_ranking_counts_3.items()}
probabilities_single_burstein_1 = {rankings: count / sum_single_burstein_1 for rankings, count in single_burstein_ranking_counts_1.items()}

# Save the results to a pickle file
with open('single_elim_burstein_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_single_burstein_8': probabilities_single_burstein_8,
        'probabilities_single_burstein_3': probabilities_single_burstein_3,
        'probabilities_single_burstein_1': probabilities_single_burstein_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 5.92 second(s)


### 17) Simulation of Single Elimination Tournament Using the Monrad Pairing System

In [ ]:
start = time.perf_counter()

single_monrad_ranking_counts_8 = permutations_8.copy()

single_monrad_ranking_counts_3 = permutations_3.copy()

single_monrad_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Monrad")

    # Create a Single Elimination tournament with the list of players and using the Monrad system pairing
    single_elim_monrad = SingleElimMulti(players, single_or_multi = "single")

    tournament_result = single_elim_monrad.simulate_the_tournament(single_elim_monrad.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    single_monrad_ranking_counts_8[tournament_result_8] += 1
    single_monrad_ranking_counts_3[tournament_result_3] += 1
    single_monrad_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_single_monrad_8 = sum(single_monrad_ranking_counts_8.values())
sum_single_monrad_3 = sum(single_monrad_ranking_counts_3.values())
sum_single_monrad_1 = sum(single_monrad_ranking_counts_1.values())

# Calculate probabilities
probabilities_single_monrad_8 = {rankings: count / sum_single_monrad_8 for rankings, count in single_monrad_ranking_counts_8.items()}
probabilities_single_monrad_3 = {rankings: count / sum_single_monrad_3 for rankings, count in single_monrad_ranking_counts_3.items()}
probabilities_single_monrad_1 = {rankings: count / sum_single_monrad_1 for rankings, count in single_monrad_ranking_counts_1.items()}

# Save the results to a pickle file
with open('single_elim_monrad_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_single_monrad_8': probabilities_single_monrad_8,
        'probabilities_single_monrad_3': probabilities_single_monrad_3,
        'probabilities_single_monrad_1': probabilities_single_monrad_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 4.23 second(s)


### 18) Simulation of Single Elimination Tournament Using the Random Pairing System

In [ ]:
start = time.perf_counter()

single_random_ranking_counts_8 = permutations_8.copy()

single_random_ranking_counts_3 = permutations_3.copy()

single_random_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Random")

    # Create a Single Elimination tournament with the list of players and using the Random system pairing
    single_elim_random = SingleElimMulti(players, single_or_multi = "single")

    tournament_result = single_elim_random.simulate_the_tournament(single_elim_random.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    single_random_ranking_counts_8[tournament_result_8] += 1
    single_random_ranking_counts_3[tournament_result_3] += 1
    single_random_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_single_random_8 = sum(single_random_ranking_counts_8.values())
sum_single_random_3 = sum(single_random_ranking_counts_3.values())
sum_single_random_1 = sum(single_random_ranking_counts_1.values())

# Calculate probabilities
probabilities_single_random_8 = {rankings: count / sum_single_random_8 for rankings, count in single_random_ranking_counts_8.items()}
probabilities_single_random_3 = {rankings: count / sum_single_random_3 for rankings, count in single_random_ranking_counts_3.items()}
probabilities_single_random_1 = {rankings: count / sum_single_random_1 for rankings, count in single_random_ranking_counts_1.items()}

# Save the results to a pickle file
with open('single_elim_random_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_single_random_8': probabilities_single_random_8,
        'probabilities_single_random_3': probabilities_single_random_3,
        'probabilities_single_random_1': probabilities_single_random_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 5.93 second(s)


### 19) Simulation of Multilevel Tournament Using the Dutch Pairing System

In [ ]:
start = time.perf_counter()

multi_dutch_ranking_counts_8 = permutations_8.copy()

multi_dutch_ranking_counts_3 = permutations_3.copy()

multi_dutch_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Dutch")

    # Create a Multilevel tournament with the list of players and using the Dutch system pairing
    multi_dutch = SingleElimMulti(players, single_or_multi = "multi")

    tournament_result = multi_dutch.simulate_the_tournament(multi_dutch.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    multi_dutch_ranking_counts_8[tournament_result_8] += 1
    multi_dutch_ranking_counts_3[tournament_result_3] += 1
    multi_dutch_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_multi_dutch_8 = sum(multi_dutch_ranking_counts_8.values())
sum_multi_dutch_3 = sum(multi_dutch_ranking_counts_3.values())
sum_multi_dutch_1 = sum(multi_dutch_ranking_counts_1.values())

# Calculate probabilities
probabilities_multi_dutch_8 = {rankings: count / sum_multi_dutch_8 for rankings, count in multi_dutch_ranking_counts_8.items()}
probabilities_multi_dutch_3 = {rankings: count / sum_multi_dutch_3 for rankings, count in multi_dutch_ranking_counts_3.items()}
probabilities_multi_dutch_1 = {rankings: count / sum_multi_dutch_1 for rankings, count in multi_dutch_ranking_counts_1.items()}

# Save the results to a pickle file
with open('multi_dutch_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_multi_dutch_8': probabilities_multi_dutch_8,
        'probabilities_multi_dutch_3': probabilities_multi_dutch_3,
        'probabilities_multi_dutch_1': probabilities_multi_dutch_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 4.39 second(s)


### 20) Simulation of Multilevel Tournament Using the Burstein Pairing System

In [ ]:
start = time.perf_counter()

multi_burstein_ranking_counts_8 = permutations_8.copy()

multi_burstein_ranking_counts_3 = permutations_3.copy()

multi_burstein_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Burstein")

    # Create a Multilevel tournament with the list of players and using the Burstein system pairing
    multi_burstein = SingleElimMulti(players, single_or_multi = "multi")

    tournament_result = multi_burstein.simulate_the_tournament(multi_burstein.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    multi_burstein_ranking_counts_8[tournament_result_8] += 1
    multi_burstein_ranking_counts_3[tournament_result_3] += 1
    multi_burstein_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_multi_burstein_8 = sum(multi_burstein_ranking_counts_8.values())
sum_multi_burstein_3 = sum(multi_burstein_ranking_counts_3.values())
sum_multi_burstein_1 = sum(multi_burstein_ranking_counts_1.values())

# Calculate probabilities
probabilities_multi_burstein_8 = {rankings: count / sum_multi_burstein_8 for rankings, count in multi_burstein_ranking_counts_8.items()}
probabilities_multi_burstein_3 = {rankings: count / sum_multi_burstein_3 for rankings, count in multi_burstein_ranking_counts_3.items()}
probabilities_multi_burstein_1 = {rankings: count / sum_multi_burstein_1 for rankings, count in multi_burstein_ranking_counts_1.items()}

# Save the results to a pickle file
with open('multi_burstein_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_multi_burstein_8': probabilities_multi_burstein_8,
        'probabilities_multi_burstein_3': probabilities_multi_burstein_3,
        'probabilities_multi_burstein_1': probabilities_multi_burstein_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 4.45 second(s)


### 21) Simulation of Multilevel Tournament Using the Monrad Pairing System

In [ ]:
start = time.perf_counter()

multi_monrad_ranking_counts_8 = permutations_8.copy()

multi_monrad_ranking_counts_3 = permutations_3.copy()

multi_monrad_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Monrad")

    # Create a Multilevel tournament with the list of players and using the Monrad system pairing
    multi_monrad = SingleElimMulti(players, single_or_multi = "multi")

    tournament_result = multi_monrad.simulate_the_tournament(multi_monrad.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    multi_monrad_ranking_counts_8[tournament_result_8] += 1
    multi_monrad_ranking_counts_3[tournament_result_3] += 1
    multi_monrad_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_multi_monrad_8 = sum(multi_monrad_ranking_counts_8.values())
sum_multi_monrad_3 = sum(multi_monrad_ranking_counts_3.values())
sum_multi_monrad_1 = sum(multi_monrad_ranking_counts_1.values())

# Calculate probabilities
probabilities_multi_monrad_8 = {rankings: count / sum_multi_monrad_8 for rankings, count in multi_monrad_ranking_counts_8.items()}
probabilities_multi_monrad_3 = {rankings: count / sum_multi_monrad_3 for rankings, count in multi_monrad_ranking_counts_3.items()}
probabilities_multi_monrad_1 = {rankings: count / sum_multi_monrad_1 for rankings, count in multi_monrad_ranking_counts_1.items()}

# Save the results to a pickle file
with open('multi_monrad_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_multi_monrad_8': probabilities_multi_monrad_8,
        'probabilities_multi_monrad_3': probabilities_multi_monrad_3,
        'probabilities_multi_monrad_1': probabilities_multi_monrad_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 5.35 second(s)


### 22) Simulation of Multilevel Tournament Using the Random Pairing System

In [ ]:
start = time.perf_counter()

multi_random_ranking_counts_8 = permutations_8.copy()

multi_random_ranking_counts_3 = permutations_3.copy()

multi_random_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 100000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Random")

    # Create a Multilevel tournament with the list of players and using the Random system pairing
    multi_random = SingleElimMulti(players, single_or_multi = "multi")

    tournament_result = multi_random.simulate_the_tournament(multi_random.single_or_multi)

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    multi_random_ranking_counts_8[tournament_result_8] += 1
    multi_random_ranking_counts_3[tournament_result_3] += 1
    multi_random_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_multi_random_8 = sum(multi_random_ranking_counts_8.values())
sum_multi_random_3 = sum(multi_random_ranking_counts_3.values())
sum_multi_random_1 = sum(multi_random_ranking_counts_1.values())

# Calculate probabilities
probabilities_multi_random_8 = {rankings: count / sum_multi_random_8 for rankings, count in multi_random_ranking_counts_8.items()}
probabilities_multi_random_3 = {rankings: count / sum_multi_random_3 for rankings, count in multi_random_ranking_counts_3.items()}
probabilities_multi_random_1 = {rankings: count / sum_multi_random_1 for rankings, count in multi_random_ranking_counts_1.items()}

# Save the results to a pickle file
with open('multi_random_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_multi_random_8': probabilities_multi_random_8,
        'probabilities_multi_random_3': probabilities_multi_random_3,
        'probabilities_multi_random_1': probabilities_multi_random_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 4.72 second(s)


### 23) Simulation of Double Elimination Tournament Using the Dutch Pairing System

In [ ]:
start = time.perf_counter()

double_dutch_ranking_counts_8 = permutations_8.copy()

double_dutch_ranking_counts_3 = permutations_3.copy()

double_dutch_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Dutch")

    # Create a Double Elimination tournament with the list of players and using the Dutch system pairing
    double_elim_dutch = DoubleElim(players)

    tournament_result = double_elim_dutch.simulate_the_tournament()

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    double_dutch_ranking_counts_8[tournament_result_8] += 1
    double_dutch_ranking_counts_3[tournament_result_3] += 1
    double_dutch_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_double_dutch_8 = sum(double_dutch_ranking_counts_8.values())
sum_double_dutch_3 = sum(double_dutch_ranking_counts_3.values())
sum_double_dutch_1 = sum(double_dutch_ranking_counts_1.values())

# Calculate probabilities
probabilities_double_dutch_8 = {rankings: count / sum_double_dutch_8 for rankings, count in double_dutch_ranking_counts_8.items()}
probabilities_double_dutch_3 = {rankings: count / sum_double_dutch_3 for rankings, count in double_dutch_ranking_counts_3.items()}
probabilities_double_dutch_1 = {rankings: count / sum_double_dutch_1 for rankings, count in double_dutch_ranking_counts_1.items()}

# Save the results to a pickle file
with open('double_elim_dutch_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_double_dutch_8': probabilities_double_dutch_8,
        'probabilities_double_dutch_3': probabilities_double_dutch_3,
        'probabilities_double_dutch_1': probabilities_double_dutch_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 7.69 second(s)


### 24) Simulation of Double Elimination Tournament Using the Burstein Pairing System

In [ ]:
start = time.perf_counter()

double_burstein_ranking_counts_8 = permutations_8.copy()

double_burstein_ranking_counts_3 = permutations_3.copy()

double_burstein_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Burstein")

    # Create a Double Elimination tournament with the list of players and using the Burstein system pairing
    double_elim_burstein = DoubleElim(players)

    tournament_result = double_elim_burstein.simulate_the_tournament()

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    double_burstein_ranking_counts_8[tournament_result_8] += 1
    double_burstein_ranking_counts_3[tournament_result_3] += 1
    double_burstein_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_double_burstein_8 = sum(double_burstein_ranking_counts_8.values())
sum_double_burstein_3 = sum(double_burstein_ranking_counts_3.values())
sum_double_burstein_1 = sum(double_burstein_ranking_counts_1.values())

# Calculate probabilities
probabilities_double_burstein_8 = {rankings: count / sum_double_burstein_8 for rankings, count in double_burstein_ranking_counts_8.items()}
probabilities_double_burstein_3 = {rankings: count / sum_double_burstein_3 for rankings, count in double_burstein_ranking_counts_3.items()}
probabilities_double_burstein_1 = {rankings: count / sum_double_burstein_1 for rankings, count in double_burstein_ranking_counts_1.items()}

# Save the results to a pickle file
with open('double_elim_burstein_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_double_burstein_8': probabilities_double_burstein_8,
        'probabilities_double_burstein_3': probabilities_double_burstein_3,
        'probabilities_double_burstein_1': probabilities_double_burstein_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 6.48 second(s)


### 25) Simulation of Double Elimination Tournament Using the Monrad Pairing System

In [ ]:
start = time.perf_counter()

double_monrad_ranking_counts_8 = permutations_8.copy()

double_monrad_ranking_counts_3 = permutations_3.copy()

double_monrad_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Monrad")

    # Create a Double Elimination tournament with the list of players and using the Monrad system pairing
    double_elim_monrad = DoubleElim(players)

    tournament_result = double_elim_monrad.simulate_the_tournament()

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    double_monrad_ranking_counts_8[tournament_result_8] += 1
    double_monrad_ranking_counts_3[tournament_result_3] += 1
    double_monrad_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_double_monrad_8 = sum(double_monrad_ranking_counts_8.values())
sum_double_monrad_3 = sum(double_monrad_ranking_counts_3.values())
sum_double_monrad_1 = sum(double_monrad_ranking_counts_1.values())

# Calculate probabilities
probabilities_double_monrad_8 = {rankings: count / sum_double_monrad_8 for rankings, count in double_monrad_ranking_counts_8.items()}
probabilities_double_monrad_3 = {rankings: count / sum_double_monrad_3 for rankings, count in double_monrad_ranking_counts_3.items()}
probabilities_double_monrad_1 = {rankings: count / sum_double_monrad_1 for rankings, count in double_monrad_ranking_counts_1.items()}

# Save the results to a pickle file
with open('double_elim_monrad_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_double_monrad_8': probabilities_double_monrad_8,
        'probabilities_double_monrad_3': probabilities_double_monrad_3,
        'probabilities_double_monrad_1': probabilities_double_monrad_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 7.41 second(s)


### 26) Simulation of Double Elimination Tournament Using the Random Pairing System

In [ ]:
start = time.perf_counter()

double_random_ranking_counts_8 = permutations_8.copy()

double_random_ranking_counts_3 = permutations_3.copy()

double_random_ranking_counts_1 = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Random")

    # Create a Double Elimination tournament with the list of players and using the random system pairing
    double_elim_random = DoubleElim(players)

    tournament_result = double_elim_random.simulate_the_tournament()

    # Convert the list of ranking to a tuple for dictionary operations
    tournament_result_8 = extract_n_players(tournament_result, 8)
    tournament_result_3 = extract_n_players(tournament_result, 3)
    tournament_result_1 = extract_n_players(tournament_result, 1)

    # Update occurrence of the resulting ranking in the list of rankings
    double_random_ranking_counts_8[tournament_result_8] += 1
    double_random_ranking_counts_3[tournament_result_3] += 1
    double_random_ranking_counts_1[tournament_result_1] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_double_random_8 = sum(double_random_ranking_counts_8.values())
sum_double_random_3 = sum(double_random_ranking_counts_3.values())
sum_double_random_1 = sum(double_random_ranking_counts_1.values())

# Calculate probabilities
probabilities_double_random_8 = {rankings: count / sum_double_random_8 for rankings, count in double_random_ranking_counts_8.items()}
probabilities_double_random_3 = {rankings: count / sum_double_random_3 for rankings, count in double_random_ranking_counts_3.items()}
probabilities_double_random_1 = {rankings: count / sum_double_random_1 for rankings, count in double_random_ranking_counts_1.items()}

# Save the results to a pickle file
with open('double_elim_random_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_double_random_8': probabilities_double_random_8,
        'probabilities_double_random_3': probabilities_double_random_3,
        'probabilities_double_random_1': probabilities_double_random_1
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 7.1 second(s)


### 27) Simulation of Double Round Robin Split Tournament Using the Dutch Pairing System

In [ ]:
start = time.perf_counter()

rr_split_dutch_simulation_results = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Dutch")

    # Create a Double Round Robin Split tournament with the list of players and using the Dutch system pairing
    rr_split_tournament_dutch = RoundRobinSplit(players)

    tournament_result = rr_split_tournament_dutch.simulate_the_tournament()

    tournament_result = extract_n_players(tournament_result, 1)

    rr_split_dutch_simulation_results[tournament_result] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_rr_split_dutch = sum(rr_split_dutch_simulation_results.values())

# Calculate probabilities
probabilities_rr_split_dutch = {rankings: count / sum_rr_split_dutch for rankings, count in rr_split_dutch_simulation_results.items()}

# Save the results to a pickle file
with open('rr_split_dutch_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_rr_split_dutch': probabilities_rr_split_dutch
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 5.57 second(s)


### 28) Simulation of Double Round Robin Split Tournament Using the Burstein Pairing System

In [ ]:
start = time.perf_counter()

rr_split_burstein_simulation_results = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Burstein")

    # Create a Double Round Robin Split tournament with the list of players and using the Burstein system pairing
    rr_split_tournament_burstein = RoundRobinSplit(players)

    tournament_result = rr_split_tournament_burstein.simulate_the_tournament()

    tournament_result = extract_n_players(tournament_result, 1)

    rr_split_burstein_simulation_results[tournament_result] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_rr_split_burstein = sum(rr_split_burstein_simulation_results.values())

# Calculate probabilities
probabilities_rr_split_burstein = {rankings: count / sum_rr_split_burstein for rankings, count in rr_split_burstein_simulation_results.items()}

# Save the results to a pickle file
with open('rr_split_burstein_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_rr_split_burstein': probabilities_rr_split_burstein
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 4.63 second(s)


### 29) Simulation of Double Round Robin Split Tournament Using the Monrad Pairing System

In [ ]:
start = time.perf_counter()

rr_split_monrad_simulation_results = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Monrad")

    # Create a Double Round Robin Split tournament with the list of players and using the Monrad system pairing
    rr_split_tournament_monrad = RoundRobinSplit(players)

    tournament_result = rr_split_tournament_monrad.simulate_the_tournament()

    tournament_result = extract_n_players(tournament_result, 1)

    rr_split_monrad_simulation_results[tournament_result] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_rr_split_monrad = sum(rr_split_monrad_simulation_results.values())

# Calculate probabilities
probabilities_rr_split_monrad = {rankings: count / sum_rr_split_monrad for rankings, count in rr_split_monrad_simulation_results.items()}

# Save the results to a pickle file
with open('rr_split_monrad_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_rr_split_monrad': probabilities_rr_split_monrad
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 5.58 second(s)


### 30) Simulation of Double Round Robin Split Tournament Using the Random Pairing System

In [ ]:
start = time.perf_counter()

rr_split_random_simulation_results = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "Random")

    # Create a Double Round Robin Split tournament with the list of players and using the Random system pairing
    rr_split_tournament_random = RoundRobinSplit(players)

    tournament_result = rr_split_tournament_random.simulate_the_tournament()

    tournament_result = extract_n_players(tournament_result, 1)

    rr_split_random_simulation_results[tournament_result] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_rr_split_random = sum(rr_split_random_simulation_results.values())

# Calculate probabilities
probabilities_rr_split_random = {rankings: count / sum_rr_split_random for rankings, count in rr_split_random_simulation_results.items()}

# Save the results to a pickle file
with open('rr_split_random_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_rr_split_random': probabilities_rr_split_random
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 5.01 second(s)


### 31) Simulation of Ladder Tournament

In [ ]:
start = time.perf_counter()

ladder_simulation_results = permutations_1.copy()

# Repeats the simulation 1000000 times
for x in range(num_of_simulations):

    # Counter
    #print(f"Counter: {x}")

    # Creating multiple players with fixed Elo scores
    players = generate_players(elo_values, "None")

    # Create a Ladder tournament with the list of players
    tournament_ladder = LadderTournament(players, rounds = 7)

    tournament_result = tournament_ladder.simulate_the_tournament(tournament_ladder.rounds)

    tournament_result = extract_n_players(tournament_result, 1)

    ladder_simulation_results[tournament_result] += 1

# Get the denominator to get the probabilities of each ranking occurring
sum_ladder = sum(ladder_simulation_results.values())

# Calculate probabilities
probabilities_ladder = {rankings: count / sum_ladder for rankings, count in ladder_simulation_results.items()}

# Save the results to a pickle file
with open('ladder_simulation_results.pickle', 'wb') as file:
    saved_data = {
        'probabilities_ladder': probabilities_ladder
    }
    pickle.dump(saved_data, file)

print("Simulation results saved successfully.")

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Simulation results saved successfully.
Finished in 2.65 second(s)


# Generating Distance Scores

### Calculating Distance Scores Between Double Round Robin and Various Tournament Formats

In [ ]:
class Distance_Scores:

    def __init__(self, basis_probabilities, comparing_probabilities):
        self.basis_probabilities = basis_probabilities # the golden standard of tournament format (Double Round Robin)
        self.comparing_probabilities = comparing_probabilities # the tournament format you want to compare

        # Extract probabilities from the dictionaries
        self.probs_p = np.array(list(self.basis_probabilities.values()))
        self.probs_q = np.array(list(self.comparing_probabilities.values()))

    # Calculate the KL-Divergence Score
    def calculate_KL_score(self):

        kl_divergence_score = entropy(self.probs_p, qk = self.probs_q)

        return kl_divergence_score

    # Calculate the Hellinger Score
    def calculate_hellinger_score(self):

        hellinger_score = np.sqrt(np.sum((np.sqrt(self.probs_p) - np.sqrt(self.probs_q)) ** 2) / 2)

        return hellinger_score

    # Calculate the Chi-Squared Statistic
    def calculate_chi_square_score(self):

        chi_square_stat, p_value = chisquare(f_obs = self.probs_q, f_exp = self.probs_p)

        return chi_square_stat

    # Calculate the Wasserstein Score
    def calculate_wasserstein_score(self):

        wasserstein_score = wasserstein_distance(self.probs_p, self.probs_q)

        return wasserstein_score

### Function to Calculate Distance Scores

In [ ]:
# Calculating various distance scores of all tournament formats wuith the Double Round Robin as the basis
def get_distance_scores(basis_probs, comparing_probs, top):

  distance_calculator = Distance_Scores(basis_probabilities = basis_probs, comparing_probabilities = comparing_probs)

  # KL-Divergence Score
  kl_score_result = distance_calculator.calculate_KL_score()
  #print(f"KL-Divergence Score: {kl_score_result}\n")

  if top == 1:
    list_of_kl_score_1.append(kl_score_result)
  elif top == 3:
    list_of_kl_score_3.append(kl_score_result)
  elif top == 8:
    list_of_kl_score_8.append(kl_score_result)

  # Hellinger Score
  hellinger_score_result = distance_calculator.calculate_hellinger_score()
  #print(f"Hellinger Score: {hellinger_score_result}\n")

  if top == 1:
    list_of_hellinger_score_1.append(hellinger_score_result)
  elif top == 3:
    list_of_hellinger_score_3.append(hellinger_score_result)
  elif top == 8:
    list_of_hellinger_score_8.append(hellinger_score_result)

  # Chi-Squared Statistic
  chi_squared_score_result = distance_calculator.calculate_chi_square_score()
  #print(f"Chi-Squared Statistic: {chi_squared_score_result}\n")

  if top == 1:
    list_of_chi_squared_score_1.append(chi_squared_score_result)
  elif top == 3:
    list_of_chi_squared_score_3.append(chi_squared_score_result)
  elif top == 8:
    list_of_chi_squared_score_8.append(chi_squared_score_result)

  # Wasserstein Score
  wasserstein_score_result = distance_calculator.calculate_wasserstein_score()
  #print(f"Wasserstein Score: {wasserstein_score_result}\n")

  if top == 1:
    list_of_wasserstein_score_1.append(wasserstein_score_result)
  elif top == 3:
    list_of_wasserstein_score_3.append(wasserstein_score_result)
  elif top == 8:
    list_of_wasserstein_score_8.append(wasserstein_score_result)

### Lists to Create a DataFrame

In [ ]:
list_of_tournament_formats = []
list_of_pairing_system = []
list_of_game_numbers = []
list_of_round_numbers = []

# Make a list to store rankings for comparison and generating of results (Top 8 Players)
list_of_ranking_comparison_8 = []

# Make a series of lists containing the results that we want to compare to the Double Round Robin (Top 8 Players)
list_of_kl_score_8 = []
list_of_hellinger_score_8 = []
list_of_chi_squared_score_8 = []
list_of_wasserstein_score_8 = []

# Make a list to store rankings for comparison and generating of results (Top 3 Players)
list_of_ranking_comparison_3 = []

# Make a series of lists containing the results that we want to compare to the Double Round Robin (Top 3 Players)
list_of_kl_score_3 = []
list_of_hellinger_score_3 = []
list_of_chi_squared_score_3 = []
list_of_wasserstein_score_3 = []

# Make a list to store rankings for comparison and generating of results (Top 1 Player)
list_of_ranking_comparison_1 = []

# Make a series of lists containing the results that we want to compare to the Double Round Robin (Top 1 Player)
list_of_kl_score_1 = []
list_of_hellinger_score_1 = []
list_of_chi_squared_score_1 = []
list_of_wasserstein_score_1 = []

### Loading the Results for Processing of Top 1 Player, Top 3 Players, and Top 8 Players

In [ ]:
# Load the saved results of Double Round Robin
with open('double_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_double_8 = loaded_data['probabilities_double_8']
probabilities_double_3 = loaded_data['probabilities_double_3']
probabilities_double_1 = loaded_data['probabilities_double_1']

list_of_ranking_comparison_8.append(probabilities_double_8)
list_of_ranking_comparison_3.append(probabilities_double_3)
list_of_ranking_comparison_1.append(probabilities_double_1)

list_of_tournament_formats.append("Double Round Robin")
list_of_pairing_system.append("NA")
list_of_game_numbers.append("56")
list_of_round_numbers.append("14")

In [ ]:
# Load the saved results of Single Round Robin
with open('single_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_single_8 = loaded_data['probabilities_single_8']
probabilities_single_3 = loaded_data['probabilities_single_3']
probabilities_single_1 = loaded_data['probabilities_single_1']

list_of_ranking_comparison_8.append(probabilities_single_8)
list_of_ranking_comparison_3.append(probabilities_single_3)
list_of_ranking_comparison_1.append(probabilities_single_1)

list_of_tournament_formats.append("Single Round Robin")
list_of_pairing_system.append("NA")
list_of_game_numbers.append("28")
list_of_round_numbers.append("7")

In [ ]:
# Load the saved results of Swiss Tournament (Dutch System) (3 Rounds)
with open('dutch_simulation_results_3.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_dutch_8 = loaded_data['probabilities_dutch_8']
probabilities_dutch_3 = loaded_data['probabilities_dutch_3']
probabilities_dutch_1 = loaded_data['probabilities_dutch_1']

list_of_ranking_comparison_8.append(probabilities_dutch_8)
list_of_ranking_comparison_3.append(probabilities_dutch_3)
list_of_ranking_comparison_1.append(probabilities_dutch_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Swiss Tournament (Burstein System) (3 Rounds)
with open('burstein_simulation_results_3.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_burstein_8 = loaded_data['probabilities_burstein_8']
probabilities_burstein_3 = loaded_data['probabilities_burstein_3']
probabilities_burstein_1 = loaded_data['probabilities_burstein_1']

list_of_ranking_comparison_8.append(probabilities_burstein_8)
list_of_ranking_comparison_3.append(probabilities_burstein_3)
list_of_ranking_comparison_1.append(probabilities_burstein_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Swiss Tournament (Monrad System) (3 Rounds)
with open('monrad_simulation_results_3.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_monrad_8 = loaded_data['probabilities_monrad_8']
probabilities_monrad_3 = loaded_data['probabilities_monrad_3']
probabilities_monrad_1 = loaded_data['probabilities_monrad_1']

list_of_ranking_comparison_8.append(probabilities_monrad_8)
list_of_ranking_comparison_3.append(probabilities_monrad_3)
list_of_ranking_comparison_1.append(probabilities_monrad_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Swiss Tournament (Random System) (3 Rounds)
with open('random_simulation_results_3.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_random_8 = loaded_data['probabilities_random_8']
probabilities_random_3 = loaded_data['probabilities_random_3']
probabilities_random_1 = loaded_data['probabilities_random_1']

list_of_ranking_comparison_8.append(probabilities_random_8)
list_of_ranking_comparison_3.append(probabilities_random_3)
list_of_ranking_comparison_1.append(probabilities_random_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Swiss Tournament (Dutch System) (4 Rounds)
with open('dutch_simulation_results_4.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_dutch_8 = loaded_data['probabilities_dutch_8']
probabilities_dutch_3 = loaded_data['probabilities_dutch_3']
probabilities_dutch_1 = loaded_data['probabilities_dutch_1']

list_of_ranking_comparison_8.append(probabilities_dutch_8)
list_of_ranking_comparison_3.append(probabilities_dutch_3)
list_of_ranking_comparison_1.append(probabilities_dutch_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("14")
list_of_round_numbers.append("4")

In [ ]:
# Load the saved results of Swiss Tournament (Burstein System) (4 Rounds)
with open('burstein_simulation_results_4.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_burstein_8 = loaded_data['probabilities_burstein_8']
probabilities_burstein_3 = loaded_data['probabilities_burstein_3']
probabilities_burstein_1 = loaded_data['probabilities_burstein_1']

list_of_ranking_comparison_8.append(probabilities_burstein_8)
list_of_ranking_comparison_3.append(probabilities_burstein_3)
list_of_ranking_comparison_1.append(probabilities_burstein_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("14")
list_of_round_numbers.append("4")

In [ ]:
# Load the saved results of Swiss Tournament (Monrad System) (4 Rounds)
with open('monrad_simulation_results_4.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_monrad_8 = loaded_data['probabilities_monrad_8']
probabilities_monrad_3 = loaded_data['probabilities_monrad_3']
probabilities_monrad_1 = loaded_data['probabilities_monrad_1']

list_of_ranking_comparison_8.append(probabilities_monrad_8)
list_of_ranking_comparison_3.append(probabilities_monrad_3)
list_of_ranking_comparison_1.append(probabilities_monrad_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("14")
list_of_round_numbers.append("4")

In [ ]:
# Load the saved results of Swiss Tournament (Random System) (4 Rounds)
with open('random_simulation_results_4.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_random_8 = loaded_data['probabilities_random_8']
probabilities_random_3 = loaded_data['probabilities_random_3']
probabilities_random_1 = loaded_data['probabilities_random_1']

list_of_ranking_comparison_8.append(probabilities_random_8)
list_of_ranking_comparison_3.append(probabilities_random_3)
list_of_ranking_comparison_1.append(probabilities_random_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("14")
list_of_round_numbers.append("4")

In [ ]:
# Load the saved results of Swiss Tournament (Dutch System) (5 Rounds)
with open('dutch_simulation_results_5.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_dutch_8 = loaded_data['probabilities_dutch_8']
probabilities_dutch_3 = loaded_data['probabilities_dutch_3']
probabilities_dutch_1 = loaded_data['probabilities_dutch_1']

list_of_ranking_comparison_8.append(probabilities_dutch_8)
list_of_ranking_comparison_3.append(probabilities_dutch_3)
list_of_ranking_comparison_1.append(probabilities_dutch_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("17")
list_of_round_numbers.append("5")

In [ ]:
# Load the saved results of Swiss Tournament (Burstein System) (5 Rounds)
with open('burstein_simulation_results_5.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_burstein_8 = loaded_data['probabilities_burstein_8']
probabilities_burstein_3 = loaded_data['probabilities_burstein_3']
probabilities_burstein_1 = loaded_data['probabilities_burstein_1']

list_of_ranking_comparison_8.append(probabilities_burstein_8)
list_of_ranking_comparison_3.append(probabilities_burstein_3)
list_of_ranking_comparison_1.append(probabilities_burstein_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("17")
list_of_round_numbers.append("5")

In [ ]:
# Load the saved results of Swiss Tournament (Monrad System) (5 Rounds)
with open('monrad_simulation_results_5.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_monrad_8 = loaded_data['probabilities_monrad_8']
probabilities_monrad_3 = loaded_data['probabilities_monrad_3']
probabilities_monrad_1 = loaded_data['probabilities_monrad_1']

list_of_ranking_comparison_8.append(probabilities_monrad_8)
list_of_ranking_comparison_3.append(probabilities_monrad_3)
list_of_ranking_comparison_1.append(probabilities_monrad_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("17")
list_of_round_numbers.append("5")

In [ ]:
# Load the saved results of Swiss Tournament (Random System) (5 Rounds)
with open('random_simulation_results_5.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_random_8 = loaded_data['probabilities_random_8']
probabilities_random_3 = loaded_data['probabilities_random_3']
probabilities_random_1 = loaded_data['probabilities_random_1']

list_of_ranking_comparison_8.append(probabilities_random_8)
list_of_ranking_comparison_3.append(probabilities_random_3)
list_of_ranking_comparison_1.append(probabilities_random_1)

list_of_tournament_formats.append("Swiss System")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("17")
list_of_round_numbers.append("5")

In [ ]:
# Load the saved results of Single Elimination Tournament (Dutch System)
with open('single_elim_dutch_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_single_dutch_8 = loaded_data['probabilities_single_dutch_8']
probabilities_single_dutch_3 = loaded_data['probabilities_single_dutch_3']
probabilities_single_dutch_1 = loaded_data['probabilities_single_dutch_1']

list_of_ranking_comparison_8.append(probabilities_single_dutch_8)
list_of_ranking_comparison_3.append(probabilities_single_dutch_3)
list_of_ranking_comparison_1.append(probabilities_single_dutch_1)

list_of_tournament_formats.append("Single Elimination")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("7")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Single Elimination Tournament (Burstein System)
with open('single_elim_burstein_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_single_burstein_8 = loaded_data['probabilities_single_burstein_8']
probabilities_single_burstein_3 = loaded_data['probabilities_single_burstein_3']
probabilities_single_burstein_1 = loaded_data['probabilities_single_burstein_1']

list_of_ranking_comparison_8.append(probabilities_single_burstein_8)
list_of_ranking_comparison_3.append(probabilities_single_burstein_3)
list_of_ranking_comparison_1.append(probabilities_single_burstein_1)

list_of_tournament_formats.append("Single Elimination")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("7")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Single Elimination Tournament (Monrad System)
with open('single_elim_monrad_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_single_monrad_8 = loaded_data['probabilities_single_monrad_8']
probabilities_single_monrad_3 = loaded_data['probabilities_single_monrad_3']
probabilities_single_monrad_1 = loaded_data['probabilities_single_monrad_1']

list_of_ranking_comparison_8.append(probabilities_single_monrad_8)
list_of_ranking_comparison_3.append(probabilities_single_monrad_3)
list_of_ranking_comparison_1.append(probabilities_single_monrad_1)

list_of_tournament_formats.append("Single Elimination")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("7")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Single Elimination Tournament (Random System)
with open('single_elim_random_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_single_random_8 = loaded_data['probabilities_single_random_8']
probabilities_single_random_3 = loaded_data['probabilities_single_random_3']
probabilities_single_random_1 = loaded_data['probabilities_single_random_1']

list_of_ranking_comparison_8.append(probabilities_single_random_8)
list_of_ranking_comparison_3.append(probabilities_single_random_3)
list_of_ranking_comparison_1.append(probabilities_single_random_1)

list_of_tournament_formats.append("Single Elimination")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("7")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Double Elimination Tournament (Dutch System)
with open('double_elim_dutch_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_double_dutch_8 = loaded_data['probabilities_double_dutch_8']
probabilities_double_dutch_3 = loaded_data['probabilities_double_dutch_3']
probabilities_double_dutch_1 = loaded_data['probabilities_double_dutch_1']

list_of_ranking_comparison_8.append(probabilities_double_dutch_8)
list_of_ranking_comparison_3.append(probabilities_double_dutch_3)
list_of_ranking_comparison_1.append(probabilities_double_dutch_1)

list_of_tournament_formats.append("Double Elimination")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("15")
list_of_round_numbers.append("6")

In [ ]:
# Load the saved results of Double Elimination Tournament (Burstein System)
with open('double_elim_burstein_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_double_burstein_8 = loaded_data['probabilities_double_burstein_8']
probabilities_double_burstein_3 = loaded_data['probabilities_double_burstein_3']
probabilities_double_burstein_1 = loaded_data['probabilities_double_burstein_1']

list_of_ranking_comparison_8.append(probabilities_double_burstein_8)
list_of_ranking_comparison_3.append(probabilities_double_burstein_3)
list_of_ranking_comparison_1.append(probabilities_double_burstein_1)

list_of_tournament_formats.append("Double Elimination")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("15")
list_of_round_numbers.append("6")

In [ ]:
# Load the saved results of Double Elimination Tournament (Monrad System)
with open('double_elim_monrad_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_double_monrad_8 = loaded_data['probabilities_double_monrad_8']
probabilities_double_monrad_3 = loaded_data['probabilities_double_monrad_3']
probabilities_double_monrad_1 = loaded_data['probabilities_double_monrad_1']

list_of_ranking_comparison_8.append(probabilities_double_monrad_8)
list_of_ranking_comparison_3.append(probabilities_double_monrad_3)
list_of_ranking_comparison_1.append(probabilities_double_monrad_1)

list_of_tournament_formats.append("Double Elimination")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("15")
list_of_round_numbers.append("6")

In [ ]:
# Load the saved results of Double Elimination Tournament (Random System)
with open('double_elim_random_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_double_random_8 = loaded_data['probabilities_double_random_8']
probabilities_double_random_3 = loaded_data['probabilities_double_random_3']
probabilities_double_random_1 = loaded_data['probabilities_double_random_1']

list_of_ranking_comparison_8.append(probabilities_double_random_8)
list_of_ranking_comparison_3.append(probabilities_double_random_3)
list_of_ranking_comparison_1.append(probabilities_double_random_1)

list_of_tournament_formats.append("Double Elimination")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("15")
list_of_round_numbers.append("6")

In [ ]:
# Load the saved results of Multilevel Tournament (Dutch System)
with open('multi_dutch_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_multi_dutch_8 = loaded_data['probabilities_multi_dutch_8']
probabilities_multi_dutch_3 = loaded_data['probabilities_multi_dutch_3']
probabilities_multi_dutch_1 = loaded_data['probabilities_multi_dutch_1']

list_of_ranking_comparison_8.append(probabilities_multi_dutch_8)
list_of_ranking_comparison_3.append(probabilities_multi_dutch_3)
list_of_ranking_comparison_1.append(probabilities_multi_dutch_1)

list_of_tournament_formats.append("Multilevel")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Multilevel Tournament (Burstein System)
with open('multi_burstein_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_multi_burstein_8 = loaded_data['probabilities_multi_burstein_8']
probabilities_multi_burstein_3 = loaded_data['probabilities_multi_burstein_3']
probabilities_multi_burstein_1 = loaded_data['probabilities_multi_burstein_1']

list_of_ranking_comparison_8.append(probabilities_multi_burstein_8)
list_of_ranking_comparison_3.append(probabilities_multi_burstein_3)
list_of_ranking_comparison_1.append(probabilities_multi_burstein_1)

list_of_tournament_formats.append("Multilevel")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Multilevel Tournament (Monrad System)
with open('multi_monrad_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_multi_monrad_8 = loaded_data['probabilities_multi_monrad_8']
probabilities_multi_monrad_3 = loaded_data['probabilities_multi_monrad_3']
probabilities_multi_monrad_1 = loaded_data['probabilities_multi_monrad_1']

list_of_ranking_comparison_8.append(probabilities_multi_monrad_8)
list_of_ranking_comparison_3.append(probabilities_multi_monrad_3)
list_of_ranking_comparison_1.append(probabilities_multi_monrad_1)

list_of_tournament_formats.append("Multilevel")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

In [ ]:
# Load the saved results of Multilevel Tournament (Random System)
with open('multi_random_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_multi_random_8 = loaded_data['probabilities_multi_random_8']
probabilities_multi_random_3 = loaded_data['probabilities_multi_random_3']
probabilities_multi_random_1 = loaded_data['probabilities_multi_random_1']

list_of_ranking_comparison_8.append(probabilities_multi_random_8)
list_of_ranking_comparison_3.append(probabilities_multi_random_3)
list_of_ranking_comparison_1.append(probabilities_multi_random_1)

list_of_tournament_formats.append("Multilevel")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("12")
list_of_round_numbers.append("3")

### Trial for Distance Scores

In [ ]:
distance_calculator = Distance_Scores(basis_probabilities = list_of_ranking_comparison_8[0], comparing_probabilities = list_of_ranking_comparison_8[1])

start = time.perf_counter()

kl_score_result = distance_calculator.calculate_KL_score()

hellinger_score_result = distance_calculator.calculate_hellinger_score()

chi_squared_score_result = distance_calculator.calculate_chi_square_score()

wasserstein_score_result = distance_calculator.calculate_wasserstein_score()

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Finished in 0.01 second(s)


### Get the Distance Scores for Top 8 Players

In [ ]:
start = time.perf_counter()

for j in list_of_ranking_comparison_8:
  get_distance_scores(basis_probs = list_of_ranking_comparison_8[0], comparing_probs = j, top = 8)

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Finished in 0.45 second(s)


### Display the Results of the Simulations (Top 8 Players)

In [ ]:
summary_df_top_8 = pd.DataFrame(columns = ["Tournament Format", "Pairing System", "Number of Games", "Number of Rounds", "Chi-Squared Statistic", "KL-Divergence Score", "Hellinger Score", "Wasserstein Score"])

summary_df_top_8["Tournament Format"] = [i for i in list_of_tournament_formats]
summary_df_top_8["Pairing System"] = [i for i in list_of_pairing_system]
summary_df_top_8["Number of Games"] = [i for i in list_of_game_numbers]
summary_df_top_8["Number of Rounds"] = [i for i in list_of_round_numbers]
summary_df_top_8["Chi-Squared Statistic"] = [i for i in list_of_chi_squared_score_8]
summary_df_top_8["KL-Divergence Score"] = [i for i in list_of_kl_score_8]
summary_df_top_8["Hellinger Score"] = [i for i in list_of_hellinger_score_8]
summary_df_top_8["Wasserstein Score"] = [i for i in list_of_wasserstein_score_8]

summary_df_top_8.to_csv("Top 8 Tournament Distance Scores.csv", index = False)

summary_df_top_8

,Tournament Format,Pairing System,Number of Games,Number of Rounds,Chi-Squared Statistic,KL-Divergence Score,Hellinger Score,Wasserstein Score
0,Double Round Robin,NA,56,14,0.000000,0.000000,0.000000,0.000000
1,Single Round Robin,NA,28,7,0.362526,0.118257,0.174422,0.000004
2,Swiss System,Dutch,12,3,11.508238,2.245491,0.607004,0.000006
3,Swiss System,Burstein,12,3,10.427903,2.102996,0.587936,0.000005
4,Swiss System,Monrad,12,3,26.799856,2.924750,0.705408,0.000008
5,Swiss System,Random,12,3,1.431754,0.603114,0.363981,0.000019
6,Swiss System,Dutch,14,4,10.143819,2.158736,0.593531,0.000004
7,Swiss System,Burstein,14,4,8.739159,2.070497,0.574879,0.000003
8,Swiss System,Monrad,14,4,24.645524,2.803353,0.682985,0.000008
9,Swiss System,Random,14,4,1.533356,0.663556,0.376449,0.000019


### Get the Distance Scores for Top 3 Players

In [ ]:
start = time.perf_counter()

for j in list_of_ranking_comparison_3:
  get_distance_scores(basis_probs = list_of_ranking_comparison_3[0], comparing_probs = j, top = 3)

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Finished in 0.03 second(s)


### Display the Results of the Simulations (Top 3 Players)

In [ ]:
summary_df_top_3 = pd.DataFrame(columns = ["Tournament Format", "Pairing System", "Number of Games", "Number of Rounds", "Chi-Squared Statistic", "KL-Divergence Score", "Hellinger Score", "Wasserstein Score"])

summary_df_top_3["Tournament Format"] = [i for i in list_of_tournament_formats]
summary_df_top_3["Pairing System"] = [i for i in list_of_pairing_system]
summary_df_top_3["Number of Games"] = [i for i in list_of_game_numbers]
summary_df_top_3["Number of Rounds"] = [i for i in list_of_round_numbers]
summary_df_top_3["Chi-Squared Statistic"] = [i for i in list_of_chi_squared_score_3]
summary_df_top_3["KL-Divergence Score"] = [i for i in list_of_kl_score_3]
summary_df_top_3["Hellinger Score"] = [i for i in list_of_hellinger_score_3]
summary_df_top_3["Wasserstein Score"] = [i for i in list_of_wasserstein_score_3]

summary_df_top_3.to_csv("Top 3 Tournament Distance Scores.csv", index = False)

summary_df_top_3

,Tournament Format,Pairing System,Number of Games,Number of Rounds,Chi-Squared Statistic,KL-Divergence Score,Hellinger Score,Wasserstein Score
0,Double Round Robin,NA,56,14,0.000000,0.000000,0.000000,0.000000
1,Single Round Robin,NA,28,7,0.176980,0.053132,0.120098,0.000656
2,Swiss System,Dutch,12,3,5.160157,0.553419,0.384798,0.002043
3,Swiss System,Burstein,12,3,3.520838,0.388694,0.323472,0.001722
4,Swiss System,Monrad,12,3,20.459319,5.397978,0.722170,0.002636
5,Swiss System,Random,12,3,5.035502,0.570793,0.390843,0.002585
6,Swiss System,Dutch,14,4,6.886649,1.955260,0.483455,0.000959
7,Swiss System,Burstein,14,4,4.359182,1.880756,0.458097,0.000535
8,Swiss System,Monrad,14,4,40.784044,5.953067,0.762461,0.001600
9,Swiss System,Random,14,4,4.497924,0.590755,0.391886,0.002557


### Additional Tournaments for Top 1 Player

In [ ]:
# Load the saved results of Double Round Robin Split (Dutch System)
with open('rr_split_dutch_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_rr_split_dutch = loaded_data['probabilities_rr_split_dutch']

list_of_ranking_comparison_1.append(probabilities_rr_split_dutch)

list_of_tournament_formats.append("Double Round Robin Split")
list_of_pairing_system.append("Dutch")
list_of_game_numbers.append("25")
list_of_round_numbers.append("7")

In [ ]:
# Load the saved results of Double Round Robin Split (Burstein System)
with open('rr_split_burstein_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_rr_split_burstein = loaded_data['probabilities_rr_split_burstein']

list_of_ranking_comparison_1.append(probabilities_rr_split_burstein)

list_of_tournament_formats.append("Double Round Robin Split")
list_of_pairing_system.append("Burstein")
list_of_game_numbers.append("25")
list_of_round_numbers.append("7")

In [ ]:
# Load the saved results of Double Round Robin Split (Monrad System)
with open('rr_split_monrad_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_rr_split_monrad = loaded_data['probabilities_rr_split_monrad']

list_of_ranking_comparison_1.append(probabilities_rr_split_monrad)

list_of_tournament_formats.append("Double Round Robin Split")
list_of_pairing_system.append("Monrad")
list_of_game_numbers.append("25")
list_of_round_numbers.append("7")

In [ ]:
# Load the saved results of Double Round Robin Split (Random System)
with open('rr_split_random_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_rr_split_random = loaded_data['probabilities_rr_split_random']

list_of_ranking_comparison_1.append(probabilities_rr_split_random)

list_of_tournament_formats.append("Double Round Robin Split")
list_of_pairing_system.append("Random")
list_of_game_numbers.append("25")
list_of_round_numbers.append("7")

In [ ]:
# Load the saved results of the Ladder Tournament
with open('ladder_simulation_results.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

probabilities_ladder = loaded_data['probabilities_ladder']

list_of_ranking_comparison_1.append(probabilities_ladder)

list_of_tournament_formats.append("Ladder")
list_of_pairing_system.append("NA")
list_of_game_numbers.append("7")
list_of_round_numbers.append("7")

### Get the Distance Scores for Top 1 Player

In [ ]:
start = time.perf_counter()

for j in list_of_ranking_comparison_1:
  get_distance_scores(basis_probs = list_of_ranking_comparison_1[0], comparing_probs = j, top = 1)

finish = time.perf_counter()

print(f"Finished in {round(finish - start, 2)} second(s)")

Finished in 0.03 second(s)


### Display the Results of the Simulations (Top 1 Player)

In [ ]:
summary_df_top_1 = pd.DataFrame(columns = ["Tournament Format", "Pairing System", "Number of Games", "Number of Rounds", "Chi-Squared Statistic", "KL-Divergence Score", "Hellinger Score", "Wasserstein Score"])

summary_df_top_1["Tournament Format"] = [i for i in list_of_tournament_formats]
summary_df_top_1["Pairing System"] = [i for i in list_of_pairing_system]
summary_df_top_1["Number of Games"] = [i for i in list_of_game_numbers]
summary_df_top_1["Number of Rounds"] = [i for i in list_of_round_numbers]
summary_df_top_1["Chi-Squared Statistic"] = [i for i in list_of_chi_squared_score_1]
summary_df_top_1["KL-Divergence Score"] = [i for i in list_of_kl_score_1]
summary_df_top_1["Hellinger Score"] = [i for i in list_of_hellinger_score_1]
summary_df_top_1["Wasserstein Score"] = [i for i in list_of_wasserstein_score_1]

summary_df_top_1.to_csv("Top 1 Tournament Distance Scores.csv", index = False)

summary_df_top_1

,Tournament Format,Pairing System,Number of Games,Number of Rounds,Chi-Squared Statistic,KL-Divergence Score,Hellinger Score,Wasserstein Score
0,Double Round Robin,NA,56,14,0.000000,0.000000,0.000000,0.000000
1,Single Round Robin,NA,28,7,0.054320,0.017769,0.069461,0.016801
2,Swiss System,Dutch,12,3,1.044350,0.220172,0.244444,0.070694
3,Swiss System,Burstein,12,3,0.627085,0.131571,0.192087,0.052228
4,Swiss System,Monrad,12,3,3.706696,0.350875,0.323853,0.089795
5,Swiss System,Random,12,3,1.405345,0.217049,0.249036,0.067812
6,Swiss System,Dutch,14,4,1.082918,0.222295,0.245901,0.070832
7,Swiss System,Burstein,14,4,0.621822,0.129462,0.190587,0.051768
8,Swiss System,Monrad,14,4,3.777005,0.355219,0.325829,0.090600
9,Swiss System,Random,14,4,1.398756,0.219577,0.250148,0.068047
